In [1]:
"""Cell 1: Setup and Imports"""
import sys
import os
import time
import warnings
import numpy as np
from IPython.display import clear_output

warnings.filterwarnings('ignore')

# Ensure src is in path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Clear all qectostim module cache BEFORE importing (fresh start)
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Also clear the bytecode cache
import importlib
importlib.invalidate_caches()

# Import testing utilities from qectostim.testing
from qectostim.testing import (
    clear_qectostim_modules,
    test_composite_construction,
    test_code_circuit,
    test_decoder_on_code,
    load_all_decoders,
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
)

# Import qectostim
from qectostim.codes import discover_all_codes
from qectostim.codes.composite import (
    ConcatenatedTopologicalCSSCode,
    HomologicalProductCode,
    DualCode,
)
from qectostim.codes.abstract_css import CSSCodeWithComplex
from qectostim.decoders import PyMatchingDecoder

# Load all available decoders
decoder_classes = load_all_decoders()

# Try to import ConcatenatedDecoder
try:
    from qectostim.decoders.concatenated_decoder import ConcatenatedDecoder
    HAS_CONCAT_DECODER = True
except ImportError as e:
    HAS_CONCAT_DECODER = False
    print(f"Note: ConcatenatedDecoder not available: {e}")

print("✓ All imports successful")
print(f"  ConcatenatedTopologicalCSSCode: {ConcatenatedTopologicalCSSCode}")
print(f"  HomologicalProductCode: {HomologicalProductCode}")
print(f"  DualCode: {DualCode}")
print(f"  Decoders loaded: {list(decoder_classes.keys())}")
print(f"  ConcatenatedDecoder available: {HAS_CONCAT_DECODER}")

✓ All imports successful
  ConcatenatedTopologicalCSSCode: <class 'qectostim.codes.composite.concatenated.ConcatenatedTopologicalCSSCode'>
  HomologicalProductCode: <class 'qectostim.codes.composite.homological_product.HomologicalProductCode'>
  DualCode: <class 'qectostim.codes.composite.dual.DualCode'>
  Decoders loaded: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'HardHierarchical', 'TurboConcatenatedTurbo', 'SoftHierarchical', 'SoftMVP', 'TurboV2', 'ExtrinsicTurbo', 'SoftMP', 'SingleShot']
  ConcatenatedDecoder available: True


In [2]:
"""Cell 2: Discover Building Block Codes"""
clear_output(wait=True)

print("="*70)
print("DISCOVERING BUILDING BLOCK CODES")
print("="*70)

# Discover codes - use smaller size for composite building
all_codes = discover_all_codes(
    max_qubits=50,  # Smaller for composition
    include_qldpc=True,
    include_subsystem=False,  # Skip subsystem for CSS operations
    include_floquet=False,    # Skip floquet for CSS operations
    include_bosonic=False,
    include_qudit=False,
    include_non_css=True,
    timeout_per_code=3.0,
)

# Filter to CSS codes only (required for concatenation and homological product)
css_codes = {}
for name, code in all_codes.items():
    if hasattr(code, 'hx') and hasattr(code, 'hz'):
        # Additional check: must have k > 0
        if code.k > 0:
            # Verify CSS condition: hx @ hz.T == 0 (mod 2)
            try:
                import numpy as np
                from scipy import sparse
                hx = code.hx
                hz = code.hz
                # Handle sparse matrices
                if sparse.issparse(hx):
                    hx = hx.toarray()
                if sparse.issparse(hz):
                    hz = hz.toarray()
                product = (hx @ hz.T) % 2
                if np.any(product != 0):
                    print(f"  Skipping {name}: hx @ hz.T != 0 (not valid CSS)")
                    continue
            except Exception as e:
                print(f"  Skipping {name}: CSS check failed ({e})")
                continue  # Skip if CSS check fails
            css_codes[name] = code

print(f"\nTotal discovered: {len(all_codes)}")
print(f"CSS codes with k>0: {len(css_codes)}")

# Categorize for smart pairing
small_codes = {}   # n <= 10, good for inner codes
medium_codes = {}  # 10 < n <= 30
surface_codes = {} # Surface-like, good for outer codes
other_codes = {}

for name, code in css_codes.items():
    n = code.n
    if 'Surface' in name or 'Toric' in name:
        surface_codes[name] = code
    elif n <= 10:
        small_codes[name] = code
    elif n <= 30:
        medium_codes[name] = code
    else:
        other_codes[name] = code

print(f"\nBy size:")
print(f"  Small (n≤10): {len(small_codes)}")
print(f"  Medium (10<n≤30): {len(medium_codes)}")
print(f"  Surface-like: {len(surface_codes)}")
print(f"  Other: {len(other_codes)}")


# List codes
print("\n" + "-"*70)
print(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
print("-"*70)
for name, code in sorted(css_codes.items(), key=lambda x: x[1].n)[:20]:
    d = code.metadata.get('distance', '?')
    print(f"{name:<40} {code.n:>4} {code.k:>3} {d:>3}")
if len(css_codes) > 20:
    print(f"... and {len(css_codes) - 20} more")

DISCOVERING BUILDING BLOCK CODES

Total discovered: 58
CSS codes with k>0: 53

By size:
  Small (n≤10): 14
  Medium (10<n≤30): 17
  Surface-like: 11
  Other: 11

----------------------------------------------------------------------
Code Name                                   n   k   d
----------------------------------------------------------------------
Repetition_3                                3   1   3
FourQubit422_[[4,2,2]]                      4   2   2
Repetition_5                                5   1   5
C6                                          6   2   2
Steane_713                                  7   1   3
Hamming_CSS_7                               7   1   3
Repetition_7                                7   1   7
TriangularColour_d3                         7   1   3
TetrahedralColorCode                        7   1   ?
Code_832                                    8   3   2
LoopToricCode4D                             8   2   ?
HexagonalColour_d2                          8   

In [3]:
# Initialize hom_results and dual_results if not defined
if 'hom_results' not in dir():
    hom_results = []
if 'dual_results' not in dir():
    dual_results = []

# Also initialize codes_with_complex if needed
if 'codes_with_complex' not in dir():
    codes_with_complex = {}

In [ ]:
"""Cell 6a: Collect Decoder Smoke Test Results (no output)

This cell collects all results for composite code decoder tests.
Results are stored in `smoke_test_results` for display in the next cell.

Structure:
1. [NO CONCATENATION] - Baseline LER for outer codes only
2. [CONCATENATED CODES] - Should have LOWER LER than baseline
3. [MULTI-LEVEL CONCATENATED] - Additional concatenation layer  
4. [BASE-BEFORE-HOMOLOGICAL-PRODUCT] - Baseline for codes before tensor product
5. [HOMOLOGICAL PRODUCT - NO METACHECKS] - Product codes without metachecks
6. [HOMOLOGICAL PRODUCT - WITH METACHECKS] - Product codes with metachecks
7. [5-CHAIN HOMOLOGICAL PRODUCTS] - SingleShot decoder test
8. [DUAL CODES] - Dual code tests
"""
import sys
from IPython.display import clear_output

# Clear qectostim modules to pick up any code fixes
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Re-import essential modules
import numpy as np
from qectostim.codes.discovery import discover_all_codes
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.codes.composite import HomologicalProductCode, DualCode
from qectostim.testing import test_decoder_on_code, load_all_decoders, STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL

# Reload decoder classes and code lists
decoder_classes = load_all_decoders()
all_codes = discover_all_codes()
css_codes = {n: c for n, c in all_codes.items() 
             if hasattr(c, 'hx') and hasattr(c, 'hz') and c.k > 0}

# Parameters
P_ERR = 0.005  # Error rate needs to be subthreshold
SHOTS = 50  # Increased shots for reliable LER estimates at low error rates
ROUNDS = 1

# Storage for all results
smoke_test_results = {
    'params': {'p': P_ERR, 'shots': SHOTS, 'rounds': ROUNDS},
    'decoder_names': [],
    'sections': {},
    'baselines': {},
    'hom_base_lers': {},
    'hom_no_meta_lers': {},
}

print("Collecting decoder smoke test results...")

if len(decoder_classes) == 0:
    print("⚠️ No decoders available")
    smoke_test_results['decoder_names'] = []
else:
    # Use all available decoders (prioritize BPOSD, PyMatching)
    primary_decoders = {}
    for name in ['BPOSD', 'PyMatching']:
        if name in decoder_classes:
            primary_decoders[name] = decoder_classes[name]
    for name, cls in decoder_classes.items():
        if name not in primary_decoders:
            primary_decoders[name] = cls
    
    decoder_names = list(primary_decoders.keys())
    smoke_test_results['decoder_names'] = decoder_names
    
    def test_code_all_decoders(code, code_label, enable_metachecks=False):
        """Test a code with ALL decoders, return dict of results."""
        results = {'label': code_label, 'no_decode': None, 'decoders': {}}
        for dec_name, dec_class in primary_decoders.items():
            try:
                res = test_decoder_on_code(
                    code, dec_class, decoder_name=dec_name,
                    code_type='CSS', p=P_ERR, shots=SHOTS, rounds=ROUNDS,
                    enable_metachecks=enable_metachecks
                )
                if results['no_decode'] is None and res.ler_no_decode is not None:
                    results['no_decode'] = res.ler_no_decode
                results['decoders'][dec_name] = {'ler': res.ler, 'status': res.status}
            except Exception as e:
                results['decoders'][dec_name] = {'ler': None, 'status': 'FAIL', 'error': str(e)[:30]}
        return results
    
    def get_best_ler(results):
        """Get best (minimum) LER from results dict."""
        lers = [r['ler'] for r in results['decoders'].values() if r.get('ler') is not None]
        return min(lers) if lers else None
    
    # =========================================================================
    # Define outer/inner code pairs for concatenation tests
    # =========================================================================
    
    # Find suitable OUTER codes (surface-like, topological)
    outer_codes = {}
    for name, code in css_codes.items():
        if ('Surface' in name or 'Rotated' in name) and code.n >= 9 and code.n <= 50:
            outer_codes[name] = code
    for name, code in css_codes.items():
        if 'Toric' in name and code.n >= 8 and code.n <= 50 and name not in outer_codes:
            outer_codes[name] = code
    if len(outer_codes) < 2:
        for name, code in sorted(css_codes.items(), key=lambda x: x[1].n):
            if 9 <= code.n <= 30 and code.k >= 1 and name not in outer_codes:
                outer_codes[name] = code
                if len(outer_codes) >= 3:
                    break
    
    # Find suitable INNER codes (small CSS codes)
    inner_codes = {}
    for name, code in css_codes.items():
        if code.n <= 7 and code.k >= 1:
            inner_codes[name] = code
        if ('Surface' in name or 'Rotated' in name) and code.n >= 9 and code.n <= 50:
            inner_codes[name] = code
    
    print(f"    Found {len(outer_codes)} outer codes: {list(outer_codes.keys())[:4]}")
    print(f"    Found {len(inner_codes)} inner codes: {list(inner_codes.keys())[:4]}")
    
    # =========================================================================
    # 1. [NO CONCATENATION] - Baseline outer codes
    # =========================================================================
    print("  [1/8] NO CONCATENATION...")
    smoke_test_results['sections']['NO_CONCAT'] = []
    
    for outer_name, outer_code in list(outer_codes.items())[:4]:
        results = test_code_all_decoders(outer_code, outer_name, enable_metachecks=False)
        best_ler = get_best_ler(results)
        if best_ler is not None:
            smoke_test_results['baselines'][outer_name] = best_ler
        results['is_baseline'] = True
        smoke_test_results['sections']['NO_CONCAT'].append(results)
    
    # =========================================================================
    # 2. [CONCATENATED CODES] - Single-level concatenation
    # =========================================================================
    print("  [2/8] CONCATENATED CODES...")
    smoke_test_results['sections']['CONCAT'] = []
    
    max_concat = 6
    concat_count = 0
    for outer_name, outer_code in list(outer_codes.items())[:3]:
        for inner_name, inner_code in list(inner_codes.items())[:3]:
            if concat_count >= max_concat:
                break
            try:
                concat = ConcatenatedTopologicalCSSCode(outer_code, inner_code)
                code_label = f"{outer_name}∘{inner_name}"
                
                results = test_code_all_decoders(concat, code_label, enable_metachecks=False)
                results['baseline_key'] = outer_name
                smoke_test_results['sections']['CONCAT'].append(results)
                concat_count += 1
                
            except Exception as e:
                smoke_test_results['sections']['CONCAT'].append({
                    'label': f"{outer_name}∘{inner_name}",
                    'error': str(e)[:50]
                })
        if concat_count >= max_concat:
            break
    
    # =========================================================================
    # 3. [MULTI-LEVEL CONCATENATED] - Double concatenation
    # =========================================================================
    print("  [3/8] MULTI-LEVEL CONCATENATED...")
    smoke_test_results['sections']['MULTI_CONCAT'] = []
    
    # Find a small inner code for multi-level
    inner_422 = None
    for name, code in css_codes.items():
        if code.n == 4 and code.k == 2:
            inner_422 = (name, code)
            break
    if inner_422 is None:
        for name, code in sorted(css_codes.items(), key=lambda x: x[1].n):
            if code.k >= 1 and code.n <= 7:
                inner_422 = (name, code)
                break
    
    # Find a surface-like outer code
    outer_surface = None
    for name, code in css_codes.items():
        if 'Surface' in name or 'Toric' in name:
            if code.n <= 25:
                outer_surface = (name, code)
                break
    if outer_surface is None:
        for name, code in sorted(css_codes.items(), key=lambda x: x[1].n):
            if code.n >= 9 and code.n <= 30:
                outer_surface = (name, code)
                break
    
    if inner_422 and outer_surface:
        inner_name, inner_code = inner_422
        outer_name, outer_code = outer_surface
        
        try:
            level1 = ConcatenatedTopologicalCSSCode(inner_code, inner_code)
            level1_label = f"({inner_name}∘{inner_name})"
            level2 = ConcatenatedTopologicalCSSCode(outer_code, level1)
            level2_label = f"{outer_name}∘{level1_label}"
            
            results = test_code_all_decoders(level2, level2_label, enable_metachecks=False)
            results['baseline_key'] = outer_name
            smoke_test_results['sections']['MULTI_CONCAT'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['MULTI_CONCAT'].append({
                'label': 'Multi-level concat',
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 4. [BASE-BEFORE-HOMOLOGICAL-PRODUCT]
    # =========================================================================
    print("  [4/8] BASE-BEFORE-HOMOLOGICAL-PRODUCT...")
    smoke_test_results['sections']['HOM_BASE'] = []
    
    codes_with_complex_local = {}
    for name, code in css_codes.items():
        if hasattr(code, 'chain_complex') and code.chain_complex is not None:
            codes_with_complex_local[name] = code
    
    try:
        from qectostim.codes.surface.toric_code_general import ToricCode
        tc = ToricCode(Lx=2, Ly=2)
        if hasattr(tc, 'chain_complex') and tc.chain_complex is not None:
            codes_with_complex_local['ToricCode_2x2'] = tc
    except:
        pass
    
    for name, code in list(codes_with_complex_local.items())[:3]:
        results = test_code_all_decoders(code, name, enable_metachecks=False)
        best_ler = get_best_ler(results)
        if best_ler is not None:
            smoke_test_results['hom_base_lers'][name] = best_ler
        results['is_baseline'] = True
        smoke_test_results['sections']['HOM_BASE'].append(results)
    
    # =========================================================================
    # 5. [HOMOLOGICAL PRODUCT - NO METACHECKS]
    # =========================================================================
    print("  [5/8] HOMOLOGICAL PRODUCT - NO METACHECKS...")
    smoke_test_results['sections']['HOM_NO_META'] = []
    
    complex_list = list(codes_with_complex_local.items())[:2]
    for name_a, code_a in complex_list:
        try:
            product = HomologicalProductCode(code_a, code_a)
            code_label = f"{name_a}⊗{name_a}"
            
            results = test_code_all_decoders(product, code_label, enable_metachecks=False)
            results['baseline_key'] = name_a
            best_ler = get_best_ler(results)
            if best_ler is not None:
                smoke_test_results['hom_no_meta_lers'][code_label] = best_ler
            smoke_test_results['sections']['HOM_NO_META'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['HOM_NO_META'].append({
                'label': f"{name_a}⊗{name_a}",
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 6. [HOMOLOGICAL PRODUCT - WITH METACHECKS]
    # =========================================================================
    print("  [6/8] HOMOLOGICAL PRODUCT - WITH METACHECKS...")
    smoke_test_results['sections']['HOM_META'] = []
    
    for name_a, code_a in complex_list:
        try:
            product = HomologicalProductCode(code_a, code_a)
            code_label = f"{name_a}⊗{name_a} (meta)"
            
            results = test_code_all_decoders(product, code_label, enable_metachecks=True)
            results['baseline_key'] = f"{name_a}⊗{name_a}"
            smoke_test_results['sections']['HOM_META'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['HOM_META'].append({
                'label': f"{name_a}⊗{name_a} (meta)",
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 7. [5-CHAIN HOMOLOGICAL PRODUCTS] - SingleShot decoder test
    # =========================================================================
    print("  [7/8] 5-CHAIN HOMOLOGICAL PRODUCTS (SingleShot test)...")
    smoke_test_results['sections']['HOM_5CHAIN'] = []
    
    five_chain_products = []
    
    try:
        from qectostim.codes.surface.toric_code_general import ToricCode
        tc_small = ToricCode(Lx=2, Ly=2)
        if hasattr(tc_small, 'chain_complex') and tc_small.chain_complex is not None:
            product_toric = HomologicalProductCode(tc_small, tc_small)
            five_chain_products.append(('ToricCode_2x2⊗ToricCode_2x2', product_toric, tc_small))
    except Exception as e:
        print(f"    ToricCode⊗ToricCode failed: {e}")
    
    for code_label, product, base_code in five_chain_products:
        has_meta_x = hasattr(product, 'meta_x') and product.meta_x is not None and product.meta_x.size > 0
        has_meta_z = hasattr(product, 'meta_z') and product.meta_z is not None and product.meta_z.size > 0
        print(f"    {code_label}: meta_x={has_meta_x}, meta_z={has_meta_z}")
        
        try:
            results = test_code_all_decoders(product, code_label, enable_metachecks=True)
            results['has_metachecks'] = has_meta_x or has_meta_z
            smoke_test_results['sections']['HOM_5CHAIN'].append(results)
        except Exception as e:
            smoke_test_results['sections']['HOM_5CHAIN'].append({
                'label': code_label,
                'error': str(e)[:50]
            })
    
    # =========================================================================
    # 8. [DUAL CODES]
    # =========================================================================
    print("  [8/8] DUAL CODES...")
    smoke_test_results['sections']['DUAL'] = []
    
    dual_candidates = []
    for name, code in css_codes.items():
        if code.n <= 15 and code.k >= 1:
            dual_candidates.append((name, code))
    
    for orig_name, orig_code in dual_candidates[:3]:
        try:
            dual = DualCode(orig_code)
            code_label = f"Dual({orig_name})"
            
            results = test_code_all_decoders(dual, code_label, enable_metachecks=False)
            smoke_test_results['sections']['DUAL'].append(results)
            
        except Exception as e:
            smoke_test_results['sections']['DUAL'].append({
                'label': f"Dual({orig_name})",
                'error': str(e)[:50]
            })

print("\n✓ Results collected. Run the next cell to display formatted output.")

    Found 11 outer codes: ['RotatedSurface_[[9,1,3]]', 'RotatedSurface_[[25,1,5]]', 'XZZX_Surface_3', 'XZZX_Surface_5']
    Found 15 inner codes: ['FourQubit422_[[4,2,2]]', 'C6', 'Steane_713', 'Hamming_CSS_7']
  [1/8] NO CONCATENATION...
  [2/8] CONCATENATED CODES...


In [ ]:
"""Cell 6b: Display Decoder Smoke Test Results

Formats and displays results collected in the previous cell.
"""
from IPython.display import clear_output
clear_output(wait=True)

import numpy as np

# Check if results exist
if 'smoke_test_results' not in dir() and 'smoke_test_results' not in globals():
    print("⚠️ Run Cell 6a first to collect results")
else:
    params = smoke_test_results['params']
    decoder_names = smoke_test_results['decoder_names']
    sections = smoke_test_results['sections']
    baselines = smoke_test_results['baselines']
    hom_base_lers = smoke_test_results['hom_base_lers']
    hom_no_meta_lers = smoke_test_results['hom_no_meta_lers']
    
    def format_ler_with_arrow(ler, baseline=None):
        """Format LER with trend arrow if baseline provided."""
        if ler is None:
            return "✗fail"
        ler_str = f"{ler:.4f}"
        if baseline is not None and baseline > 0:
            if ler < baseline * 0.5:
                return f"{ler_str} ▼▼"
            elif ler < baseline * 0.9:
                return f"{ler_str} ▼"
            elif ler > baseline * 1.5:
                return f"{ler_str} ▲▲!"
            elif ler > baseline * 1.1:
                return f"{ler_str} ▲"
        return ler_str
    
    def get_best_ler(results):
        """Get best LER from results."""
        if 'decoders' not in results:
            return None
        lers = [r['ler'] for r in results['decoders'].values() if r.get('ler') is not None]
        return min(lers) if lers else None
    
    # Print header
    print("=" * 120)
    print("DECODER SMOKE TEST ON COMPOSITE CODES - COMPREHENSIVE COMPARISON")
    print("=" * 120)
    print(f"\nParameters: p={params['p']}, shots={params['shots']}, rounds={params['rounds']}")
    print("Expectation: More concatenation → LOWER LER; Metachecks → LOWER LER for homological products\n")
    print(f"Testing with decoders: {decoder_names}\n")
    
    # Build header row
    col_width = 12
    code_col_width = 50
    header = f"  {'Code':<{code_col_width}} {'no-dec':>{col_width}}"
    for dec_name in decoder_names:
        header += f" {dec_name[:col_width]:<{col_width}}"
    header += "  Status"
    sep_line = "-" * (code_col_width + 4 + col_width + (col_width + 1) * len(decoder_names) + 15)
    
    print(header)
    print(sep_line)
    
    # Section display info
    section_info = [
        ('NO_CONCAT', '[NO CONCATENATION] - Baseline LER for outer codes', None),
        ('CONCAT', '[CONCATENATED CODES] - Should have LOWER LER than outer code baseline', 'baselines'),
        ('MULTI_CONCAT', '[MULTI-LEVEL CONCATENATED] - Double concatenation (even lower LER expected)', 'baselines'),
        ('HOM_BASE', '[BASE-BEFORE-HOMOLOGICAL-PRODUCT] - Baseline codes for tensor product', None),
        ('HOM_NO_META', '[HOMOLOGICAL PRODUCT - NO METACHECKS] - 4D codes without metachecks', 'hom_base_lers'),
        ('HOM_META', '[HOMOLOGICAL PRODUCT - WITH METACHECKS] - Should be BETTER than no metachecks', 'hom_no_meta_lers'),
        ('HOM_5CHAIN', '[5-CHAIN HOMOLOGICAL PRODUCTS] - ToricCode⊗ToricCode etc. for SingleShot decoder', None),
        ('DUAL', '[DUAL CODES]', None),
    ]
    
    all_section_lers = {}
    
    for sec_key, sec_title, baseline_source in section_info:
        if sec_key not in sections or not sections[sec_key]:
            continue
        
        print(f"\n{sec_title}")
        all_section_lers[sec_key] = []
        
        for results in sections[sec_key]:
            label = results.get('label', '???')
            if len(label) > code_col_width - 2:
                label = label[:code_col_width - 5] + "..."
            
            # Handle error case
            if 'error' in results:
                print(f"  {label:<{code_col_width}} ERROR: {results['error']}")
                continue
            
            # Get baseline for comparison
            baseline = None
            if baseline_source == 'baselines':
                baseline = baselines.get(results.get('baseline_key'))
            elif baseline_source == 'hom_base_lers':
                baseline = hom_base_lers.get(results.get('baseline_key'))
            elif baseline_source == 'hom_no_meta_lers':
                baseline = hom_no_meta_lers.get(results.get('baseline_key'))
            
            # No-decode column
            no_dec = results.get('no_decode')
            no_dec_str = f"{no_dec:.4f}" if no_dec is not None else "---"
            
            row = f"  {label:<{code_col_width}} {no_dec_str:>{col_width}}"
            
            # Each decoder column
            for dec_name in decoder_names:
                dec_res = results.get('decoders', {}).get(dec_name, {})
                ler = dec_res.get('ler')
                if dec_res.get('status') == 'SKIP':
                    row += f" {'-skip':<{col_width}}"
                else:
                    ler_str = format_ler_with_arrow(ler, baseline)
                    row += f" {ler_str:<{col_width}}"
            
            # Status column
            best_ler = get_best_ler(results)
            if best_ler is not None:
                all_section_lers[sec_key].append(best_ler)
            
            is_baseline = results.get('is_baseline', False)
            if is_baseline:
                row += "  baseline"
            elif baseline is not None and best_ler is not None:
                if best_ler < baseline * 0.3:
                    row += f"  {STATUS_OK} MUCH better"
                elif best_ler < baseline * 0.7:
                    row += f"  {STATUS_OK} improved"
                elif best_ler < baseline * 0.95:
                    row += f"  {STATUS_OK} slightly better"
                elif best_ler > baseline * 1.5:
                    row += f"  {STATUS_FAIL} WORSE!"
                elif best_ler > baseline * 1.1:
                    row += f"  {STATUS_WARN} worse"
                else:
                    row += f"  ~ similar"
            else:
                row += "  ---"
            
            print(row)
    
    print(sep_line)
    
    # SUMMARY
    print("\n" + "=" * 80)
    print("SUMMARY")
    print("=" * 80)
    
    print(f"\n{'Section':<40} {'Avg LER':>12} {'Min LER':>12} {'Max LER':>12} {'Count':>8}")
    for sec_key in ['NO_CONCAT', 'CONCAT', 'MULTI_CONCAT', 'HOM_BASE', 'HOM_NO_META', 'HOM_META', 'HOM_5CHAIN', 'DUAL']:
        if sec_key in all_section_lers and all_section_lers[sec_key]:
            lers = all_section_lers[sec_key]
            avg_ler = np.mean(lers)
            min_ler = np.min(lers)
            max_ler = np.max(lers)
            print(f"{sec_key:<40} {avg_ler:>12.4f} {min_ler:>12.4f} {max_ler:>12.4f} {len(lers):>8}")
    
    # Key comparisons
    print("\nKey Comparisons:")
    
    if 'NO_CONCAT' in all_section_lers and 'CONCAT' in all_section_lers:
        base_avg = np.mean(all_section_lers['NO_CONCAT'])
        concat_avg = np.mean(all_section_lers['CONCAT'])
        if base_avg > 0:
            improvement = (base_avg - concat_avg) / base_avg * 100
            status = STATUS_OK if improvement > 20 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Concatenation: {base_avg:.4f} → {concat_avg:.4f} ({improvement:+.1f}%)")
    
    if 'NO_CONCAT' in all_section_lers and 'MULTI_CONCAT' in all_section_lers:
        base_avg = np.mean(all_section_lers['NO_CONCAT'])
        multi_avg = np.mean(all_section_lers['MULTI_CONCAT'])
        if base_avg > 0:
            improvement = (base_avg - multi_avg) / base_avg * 100
            status = STATUS_OK if improvement > 40 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Multi-concat: {base_avg:.4f} → {multi_avg:.4f} ({improvement:+.1f}%)")
    
    if 'HOM_NO_META' in all_section_lers and 'HOM_META' in all_section_lers:
        no_meta_avg = np.mean(all_section_lers['HOM_NO_META'])
        meta_avg = np.mean(all_section_lers['HOM_META'])
        if no_meta_avg > 0:
            improvement = (no_meta_avg - meta_avg) / no_meta_avg * 100
            status = STATUS_OK if improvement > 10 else (STATUS_WARN if improvement > 0 else STATUS_FAIL)
            print(f"  {status} Metachecks (hom. product): {no_meta_avg:.4f} → {meta_avg:.4f} ({improvement:+.1f}%)")
        elif meta_avg == 0 and no_meta_avg == 0:
            print(f"  {STATUS_OK} Metachecks (hom. product): both 0.0000 (perfect)")
    
    print("\n" + "=" * 80)

DECODER SMOKE TEST ON COMPOSITE CODES - COMPREHENSIVE COMPARISON

Parameters: p=0.005, shots=10000, rounds=1
Expectation: More concatenation → LOWER LER; Metachecks → LOWER LER for homological products

Testing with decoders: ['BPOSD', 'PyMatching', 'FusionBlossom', 'BeliefMatching', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'HardHierarchical', 'TurboConcatenatedTurbo', 'SoftHierarchical', 'SoftMVP', 'TurboV2', 'ExtrinsicTurbo', 'SoftMP', 'SingleShot']

  Code                                                     no-dec BPOSD        PyMatching   FusionBlosso BeliefMatchi Tesseract    UnionFind    MLE          Hypergraph   Chromobius   Concatenated FlatConcat   HardHierarch TurboConcate SoftHierarch SoftMVP      TurboV2      ExtrinsicTur SoftMP       SingleShot    Status
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
"""Cell 7: Test hierarchical decoders via testing_utils framework"""
import sys
# Clear cached modules to pick up fixes
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

import numpy as np
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.small.four_two_two import FourQubit422Code
from qectostim.codes.small.steane_713 import SteaneCode713
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.testing import test_decoder_on_code, load_all_decoders

print("=" * 70)
print("DIAGNOSTIC: Testing hierarchical decoders on concatenated code")
print("=" * 70)

# Build the code
inner = SteaneCode713()
outer = RotatedSurfaceCode(distance=3)  # Smaller for faster testing
concat = ConcatenatedTopologicalCSSCode(outer=outer, inner=inner)

print(f"\nCode: {concat.name}")
print(f"  Inner: n={inner.n}, k={inner.k}, hx={inner.hx.shape}, hz={inner.hz.shape}")
print(f"  Outer: n={outer.n}, k={outer.k}")
print(f"  Total: n={concat.n}, k={concat.k}")

# Test parameters
P_ERR = 0.005
SHOTS = 1000
ROUNDS = 1

# Load all decoders
decoder_classes = load_all_decoders()
print(f"\nLoaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

# Test the hierarchical decoders + baselines
decoders_to_test = ['BPOSD', 'BeliefMatching', 'TurboV2', 'ExtrinsicTurbo', 'SoftHierarchical', 'SoftMP', 'HardHierarchical']

print(f"\nTesting on: {concat.name}")
print(f"Parameters: p={P_ERR}, shots={SHOTS}, rounds={ROUNDS}")
print("-"*70)

results_summary = {}
for dec_name in decoders_to_test:
    if dec_name not in decoder_classes:
        print(f"  {dec_name}: NOT LOADED")
        continue
    
    dec_class = decoder_classes[dec_name]
    result = test_decoder_on_code(
        code=concat, 
        decoder_class=dec_class,
        decoder_name=dec_name,
        p=P_ERR, 
        shots=SHOTS, 
        rounds=ROUNDS
    )
    
    status_icon = "✓" if result.status == "OK" else ("⚠" if result.status == "WARN" else "✗")
    print(f"  {dec_name}: {status_icon} {result.status}")
    if result.ler is not None:
        print(f"    LER: {result.ler:.4f}")
        results_summary[dec_name] = result.ler
    if result.warnings:
        for w in result.warnings[:2]:  # Limit warnings
            print(f"    Warning: {w[:80]}")
    if result.error:
        print(f"    Error: {result.error[:80]}")

# Summary comparison
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
if 'BPOSD' in results_summary:
    bposd_ler = results_summary['BPOSD']
    print(f"BPOSD baseline: {bposd_ler:.4f}")
    print("\nRelative to BPOSD:")
    for name, ler in results_summary.items():
        if name != 'BPOSD':
            ratio = ler / bposd_ler if bposd_ler > 0 else float('inf')
            status = "✓" if ratio < 2 else ("⚠" if ratio < 10 else "✗")
            print(f"  {status} {name}: {ler:.4f} ({ratio:.1f}x BPOSD)")

DIAGNOSTIC: Testing hierarchical decoders on concatenated code

Code: Concatenated(RotatedSurfaceCode, SteaneCode713)
  Inner: n=7, k=1, hx=(3, 7), hz=(3, 7)
  Outer: n=9, k=1
  Total: n=63, k=1

Loaded 19 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'HardHierarchical', 'TurboConcatenatedTurbo', 'SoftHierarchical', 'SoftMVP', 'TurboV2', 'ExtrinsicTurbo', 'SoftMP', 'SingleShot']

Testing on: Concatenated(RotatedSurfaceCode, SteaneCode713)
Parameters: p=0.005, shots=1000, rounds=1
----------------------------------------------------------------------
  BPOSD: ✓ OK
    LER: 0.0030
  BeliefMatching: ✗ SKIP
    Error: Hyperedge incompatible
  TurboV2: ✓ OK
    LER: 0.1590
  ExtrinsicTurbo: ✓ OK
    LER: 0.1470


/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/decoders/soft_hierarchical_decoder.py:190: UserWarning: SoftHierarchicalDecoder: DEM has 1 observables, expected 10 (1 main + 9 inner blocks). Using code-based soft information estimation (may be less accurate).
  warnings.warn(
/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/my_venv/lib/python3.11/site-packages/ldpc/_legacy_ldpc_v1/_legacy_bposd_decoder.py:45: UserWarning: This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  warnings.warn(


  SoftHierarchical: ✓ OK
    LER: 0.2920
  SoftMP: ✓ OK
    LER: 0.1540
  HardHierarchical: ✓ OK
    LER: 0.1050

SUMMARY
BPOSD baseline: 0.0030

Relative to BPOSD:
  ✗ TurboV2: 0.1590 (53.0x BPOSD)
  ✗ ExtrinsicTurbo: 0.1470 (49.0x BPOSD)
  ✗ SoftHierarchical: 0.2920 (97.3x BPOSD)
  ✗ SoftMP: 0.1540 (51.3x BPOSD)
  ✗ HardHierarchical: 0.1050 (35.0x BPOSD)


/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/decoders/hard_hierarchical_decoder.py:229: UserWarning: HierarchicalConcatenatedDecoder: DEM has 1 observables, expected 10 (1 main + 9 inner blocks). Inner code type: SteaneCode713, Multi-level: False
  warnings.warn(
/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/decoders/hard_hierarchical_decoder.py:492: UserWarning: HierarchicalConcatenatedDecoder: Using pre-calculated P_logical = 0.0220 for outer decoder edge weights. This is a hard hierarchical approach.
  warnings.warn(


In [ ]:
"""Cell 8: Debug and test TurboV2/ExtrinsicTurbo directly"""
import sys
# Clear cached modules
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

import numpy as np
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.small.four_two_two import FourQubit422Code
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.testing import test_decoder_on_code
from qectostim.decoders.turbo_decoder_v2 import TurboDecoderV2
from qectostim.decoders.extrinsic_turbo_decoder import ExtrinsicTurboDecoder

print("=" * 70)
print("Testing TurboV2 and ExtrinsicTurbo directly")
print("=" * 70)

# Build the code
inner = FourQubit422Code()
outer = RotatedSurfaceCode(distance=3)
concat = ConcatenatedTopologicalCSSCode(outer=outer, inner=inner)

print(f"\nCode: {concat.name}")
print(f"  n={concat.n}, k={concat.k}")

# Test parameters
P_ERR = 0.005
SHOTS = 1000
ROUNDS = 1

# Direct test of TurboV2 
print("\n--- Testing TurboV2 ---")
result = test_decoder_on_code(
    code=concat, 
    decoder_class=TurboDecoderV2,
    decoder_name='TurboV2',
    p=P_ERR, 
    shots=SHOTS, 
    rounds=ROUNDS
)
print(f"  Status: {result.status}")
if result.ler is not None:
    print(f"  LER: {result.ler:.4f}")
if result.error:
    print(f"  Error: {result.error}")
if result.warnings:
    for w in result.warnings[:3]:
        print(f"  Warning: {w[:100]}")

# Direct test of ExtrinsicTurbo
print("\n--- Testing ExtrinsicTurbo ---")
result = test_decoder_on_code(
    code=concat, 
    decoder_class=ExtrinsicTurboDecoder,
    decoder_name='ExtrinsicTurbo',
    p=P_ERR, 
    shots=SHOTS, 
    rounds=ROUNDS
)
print(f"  Status: {result.status}")
if result.ler is not None:
    print(f"  LER: {result.ler:.4f}")
if result.error:
    print(f"  Error: {result.error}")
if result.warnings:
    for w in result.warnings[:3]:
        print(f"  Warning: {w[:100]}")

Testing TurboV2 and ExtrinsicTurbo directly

Code: Concatenated(RotatedSurfaceCode, FourQubit422Code)
  n=36, k=10

--- Testing TurboV2 ---
  Status: OK
  LER: 0.0840

--- Testing ExtrinsicTurbo ---
  Status: OK
  LER: 0.0940


In [ ]:
"""Cell 9: Compare test_decoder_on_code results at different error rates"""
import sys
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

import numpy as np
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.small.four_two_two import FourQubit422Code
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.testing import test_decoder_on_code
from qectostim.decoders.soft_message_passing_decoder import SoftMessagePassingDecoder
from qectostim.decoders.bp_osd import BPOSDDecoder

print("=" * 70)
print("COMPARISON: SoftMP vs BPOSD at different error rates")
print("=" * 70)

# Build the code
inner = FourQubit422Code()
outer = RotatedSurfaceCode(distance=3)
concat = ConcatenatedTopologicalCSSCode(outer=outer, inner=inner)

print(f"\nCode: {concat.name}")
print(f"  [[{concat.n}, {concat.k}]]")

# Test at multiple error rates
SHOTS = 2000
ROUNDS = 1

print(f"\nTesting with shots={SHOTS}, rounds={ROUNDS}")
print("-" * 70)
print(f"{'p':>8} | {'BPOSD':>10} | {'SoftMP':>10} | {'Ratio':>8}")
print("-" * 70)

for p in [0.001, 0.003, 0.005, 0.01, 0.02]:
    # Test BPOSD
    bposd_result = test_decoder_on_code(
        code=concat, decoder_class=BPOSDDecoder, decoder_name='BPOSD',
        p=p, shots=SHOTS, rounds=ROUNDS
    )
    
    # Test SoftMP
    softmp_result = test_decoder_on_code(
        code=concat, decoder_class=SoftMessagePassingDecoder, decoder_name='SoftMP',
        p=p, shots=SHOTS, rounds=ROUNDS
    )
    
    bposd_ler = bposd_result.ler if bposd_result.ler is not None else float('nan')
    softmp_ler = softmp_result.ler if softmp_result.ler is not None else float('nan')
    
    if bposd_ler > 0 and softmp_ler > 0:
        ratio = softmp_ler / bposd_ler
        ratio_str = f"{ratio:.1f}x"
    elif bposd_ler == 0 and softmp_ler == 0:
        ratio_str = "both 0"
    elif bposd_ler == 0:
        ratio_str = "∞"
    else:
        ratio_str = "n/a"
    
    print(f"{p:>8.3f} | {bposd_ler:>10.4f} | {softmp_ler:>10.4f} | {ratio_str:>8}")

print("-" * 70)

COMPARISON: SoftMP vs BPOSD at different error rates

Code: Concatenated(RotatedSurfaceCode, FourQubit422Code)
  [[36, 10]]

Testing with shots=2000, rounds=1
----------------------------------------------------------------------
       p |      BPOSD |     SoftMP |    Ratio
----------------------------------------------------------------------
   0.001 |     0.0000 |     0.0180 |        ∞
   0.003 |     0.0010 |     0.0605 |    60.5x
   0.005 |     0.0025 |     0.1135 |    45.4x
   0.010 |     0.0110 |     0.1695 |    15.4x
   0.020 |     0.0535 |     0.3220 |     6.0x
----------------------------------------------------------------------


In [ ]:
"""Cell 10: Diagnose WHY SoftMP fails - look at soft info computation"""
import sys
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

import numpy as np
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.small.four_two_two import FourQubit422Code
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
from qectostim.decoders.soft_message_passing_decoder import SoftMessagePassingDecoder
from qectostim.decoders.block_extraction import BlockExtractor, collapse_block_syndrome

print("=" * 70)
print("DIAGNOSIS: Why does SoftMP fail?")
print("=" * 70)

# Build the code
inner = FourQubit422Code()
outer = RotatedSurfaceCode(distance=3)
concat = ConcatenatedTopologicalCSSCode(outer=outer, inner=inner)

# Generate circuit and DEM
noise = CircuitDepolarizingNoise(p1=0.01, p2=0.01)
exp = CSSMemoryExperiment(code=concat, rounds=1, noise_model=noise)
circuit = exp.to_stim()
dem = circuit.detector_error_model()

# Create decoder
decoder = SoftMessagePassingDecoder(concat, dem, rounds=1, basis="Z")

print(f"\nCode: [[{concat.n}, {concat.k}]]")
print(f"DEM: {dem.num_detectors} detectors, {dem.num_observables} observables")
print(f"Blocks: {decoder.n_blocks}, dets_per_block: {decoder.dets_per_block}")

# Check inner logical operators
print(f"\nInner logical operators in decoder:")
print(f"  inner_lz: {decoder.inner_lz}")
print(f"  inner_lx: {decoder.inner_lx}")

# Sample
sampler = circuit.compile_detector_sampler()
samples, obs = sampler.sample(shots=500, separate_observables=True)
nonzero_mask = np.sum(samples, axis=1) > 0
print(f"\nSampled 500 shots, {np.sum(nonzero_mask)} with nonzero syndrome")

# Decode and find errors
preds = decoder.decode_batch(samples)
wrong = (preds[:, 0] != obs[:, 0])
print(f"SoftMP errors: {np.sum(wrong)}/{len(samples)} = {np.sum(wrong)/len(samples)*100:.1f}%")

# Analyze the SOFT INFO computation
print("\n" + "=" * 70)
print("ANALYZING SOFT INFO COMPUTATION")
print("=" * 70)

# The key question: How does SoftMP compute P(logical_error | syndrome)?
# It does:
# 1. For each block: collapse detector syndrome to stabilizer syndrome
# 2. Compute soft info for block's logical error
# 3. Combine soft info across blocks using outer code structure

# Let's trace a specific error case
fail_indices = np.where(wrong)[0]
if len(fail_indices) > 0:
    idx = fail_indices[0]
    syn = samples[idx]
    true_obs = obs[idx, 0]
    pred_obs = preds[idx, 0]
    
    print(f"\nAnalyzing failing shot {idx}:")
    print(f"  True observable: {true_obs}")
    print(f"  SoftMP prediction: {pred_obs}")
    print(f"  Total syndrome weight: {np.sum(syn)}")
    
    # Show active blocks
    print(f"\n  Active blocks:")
    for blk in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[blk]
        block_syn = syn[det_start:det_stop]
        if np.sum(block_syn) > 0:
            stab_syn = collapse_block_syndrome(
                block_syn, decoder.n_inner_x_checks, decoder.n_inner_z_checks, 1, "Z"
            )
            print(f"    Block {blk}: det={block_syn}, stab={stab_syn}")
    
    # Show outer syndrome
    outer_start, outer_end = decoder.outer_slice
    outer_syn = syn[outer_start:outer_end]
    print(f"  Outer syndrome: weight={np.sum(outer_syn)}")

# KEY INSIGHT: What's the [[4,2,2]] inner code doing?
print("\n" + "=" * 70)
print("KEY INSIGHT: The [[4,2,2]] inner code problem")
print("=" * 70)

print(f"""
The [[4,2,2]] code has:
  - 4 physical qubits
  - 2 logical qubits (k=2)
  - Distance 2

The problem: We're encoding a SINGLE outer qubit into k=2 inner logical qubits.
The decoder doesn't know which of the 2 inner logicals corresponds to the outer qubit!

For Z-basis decoding:
  - Inner code has 1 Z-stabilizer
  - Inner code has 2 logical Z operators (lz1, lz2)
  - The decoder needs to track errors on the correct logical qubit
  
If decoder.inner_lz = {decoder.inner_lz}
This means it's tracking errors on physical qubits {np.where(decoder.inner_lz)[0] if decoder.inner_lz is not None else 'None'}

For a [[4,2,2]] code, the logical Z operators should be on disjoint qubit pairs.
If we're tracking the WRONG logical operator, we'll decode incorrectly 50% of the time!
""")

# Check what inner_lz actually is
print(f"\nActual inner logical Z support:")
print(f"  decoder.inner_lz: {decoder.inner_lz}")

# This should be the logical Z of the inner code restricted to one of its k logical qubits
# For [[4,2,2]], typical lz = [[1,1,0,0], [0,0,1,1]]
print(f"\nInner code hx: {inner.hx}")
print(f"Inner code hz: {inner.hz}")

DIAGNOSIS: Why does SoftMP fail?

Code: [[36, 10]]
DEM: 26 detectors, 1 observables
Blocks: 9, dets_per_block: 2

Inner logical operators in decoder:
  inner_lz: [1 0 1 0]
  inner_lx: [1 1 0 0]

Sampled 500 shots, 0 with nonzero syndrome
SoftMP errors: 0/500 = 0.0%

ANALYZING SOFT INFO COMPUTATION

KEY INSIGHT: The [[4,2,2]] inner code problem

The [[4,2,2]] code has:
  - 4 physical qubits
  - 2 logical qubits (k=2)
  - Distance 2

The problem: We're encoding a SINGLE outer qubit into k=2 inner logical qubits.
The decoder doesn't know which of the 2 inner logicals corresponds to the outer qubit!

For Z-basis decoding:
  - Inner code has 1 Z-stabilizer
  - Inner code has 2 logical Z operators (lz1, lz2)
  - The decoder needs to track errors on the correct logical qubit

If decoder.inner_lz = [1 0 1 0]
This means it's tracking errors on physical qubits [0 2]

For a [[4,2,2]] code, the logical Z operators should be on disjoint qubit pairs.
If we're tracking the WRONG logical operator, we'

In [ ]:
"""
HYPOTHESIS TEST (v2): k=1 vs k=2 inner codes
=============================================
After fixing string-format logical operator parsing.

Theory: Hierarchical decoders should work well for k=1 inner codes but
poorly for k>1 inner codes due to ambiguity in which logical qubit was corrupted.
"""
import sys
import numpy as np

# Reload modules to get fresh state (picks up fix to _get_logical_support)
modules_to_clear = [m for m in sys.modules if 'qectostim' in m]
for m in modules_to_clear:
    del sys.modules[m]

from qectostim.codes.composite import ConcatenatedTopologicalCSSCode
from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.codes.small import FourQubit422Code, RepetitionCode, SteaneCode713
from qectostim.decoders import BPOSDDecoder
from qectostim.decoders.soft_message_passing_decoder import SoftMessagePassingDecoder
from qectostim.testing import test_decoder_on_code

# Test parameters
P_ERR = 0.01
SHOTS = 2000
ROUNDS = 1

print("=" * 70)
print("HYPOTHESIS: k=1 inner codes should match BPOSD, k=2 should be worse")
print("(v2: with string-format logical operator fix)")
print("=" * 70)

outer = RotatedSurfaceCode(distance=3)
inner_codes = {
    "FourQubit422 [[4,2,2]] k=2": FourQubit422Code(),
    "RepetitionCode [[3,1,3]] k=1": RepetitionCode(N=3),
    "SteaneCode [[7,1,3]] k=1": SteaneCode713(),
}

results = {}

for name, inner in inner_codes.items():
    print(f"\n{'='*70}")
    print(f"Testing: {name}")
    print(f"Inner code: n={inner.n}, k={inner.k}, d={inner.distance if hasattr(inner, 'distance') else '?'}")
    print("=" * 70)
    
    # Build concatenated code
    try:
        concat = ConcatenatedTopologicalCSSCode(outer, inner)
        print(f"Concatenated code: [[{concat.n}, {concat.k}]]")
    except Exception as e:
        print(f"  FAILED to build concatenated code: {e}")
        results[name] = {"bposd": None, "softmp": None, "ratio": None, "k": inner.k}
        continue
    
    # Test BPOSD (baseline)
    try:
        bposd_result = test_decoder_on_code(
            code=concat, decoder_class=BPOSDDecoder, decoder_name='BPOSD',
            p=P_ERR, shots=SHOTS, rounds=ROUNDS
        )
        bposd_ler = bposd_result.ler if bposd_result.ler is not None else float('nan')
        print(f"  BPOSD:  LER = {bposd_ler:.4f}")
    except Exception as e:
        print(f"  BPOSD:  FAILED - {e}")
        bposd_ler = None
    
    # Test SoftMP hierarchical decoder
    try:
        softmp_result = test_decoder_on_code(
            code=concat, decoder_class=SoftMessagePassingDecoder, decoder_name='SoftMP',
            p=P_ERR, shots=SHOTS, rounds=ROUNDS
        )
        softmp_ler = softmp_result.ler if softmp_result.ler is not None else float('nan')
        print(f"  SoftMP: LER = {softmp_ler:.4f}")
    except Exception as e:
        print(f"  SoftMP: FAILED - {e}")
        softmp_ler = None
    
    # Calculate ratio
    if bposd_ler is not None and softmp_ler is not None and bposd_ler > 0:
        ratio = softmp_ler / bposd_ler
        print(f"  Ratio (SoftMP/BPOSD): {ratio:.1f}x")
    elif bposd_ler == 0:
        ratio = float('inf') if softmp_ler > 0 else 1.0
        print(f"  Ratio: BPOSD=0, SoftMP={softmp_ler:.4f if softmp_ler else 'N/A'}")
    else:
        ratio = None
    
    results[name] = {
        "bposd": bposd_ler,
        "softmp": softmp_ler,
        "ratio": ratio,
        "k": inner.k
    }

# Summary
print("\n" + "=" * 70)
print("SUMMARY: Does k=1 perform better than k=2?")
print("=" * 70)
print(f"{'Inner Code':<35} {'k':>3} {'BPOSD':>8} {'SoftMP':>8} {'Ratio':>8}")
print("-" * 70)

for name, data in results.items():
    bposd_str = f"{data['bposd']:.4f}" if data['bposd'] is not None and not np.isnan(data['bposd']) else "FAIL"
    softmp_str = f"{data['softmp']:.4f}" if data['softmp'] is not None and not np.isnan(data['softmp']) else "FAIL"
    ratio_str = f"{data['ratio']:.1f}x" if data['ratio'] is not None and data['ratio'] != float('inf') else "N/A"
    print(f"{name:<35} {data['k']:>3} {bposd_str:>8} {softmp_str:>8} {ratio_str:>8}")

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
k1_ratios = [d['ratio'] for n, d in results.items() if d['k'] == 1 and d['ratio'] is not None and d['ratio'] != float('inf')]
k2_ratios = [d['ratio'] for n, d in results.items() if d['k'] == 2 and d['ratio'] is not None and d['ratio'] != float('inf')]

if k1_ratios:
    avg_k1 = np.mean(k1_ratios)
    print(f"Average ratio for k=1 codes: {avg_k1:.1f}x")
if k2_ratios:
    avg_k2 = np.mean(k2_ratios)
    print(f"Average ratio for k=2 codes: {avg_k2:.1f}x")

if k1_ratios and k2_ratios:
    if avg_k1 < avg_k2 / 2:
        print("✓ HYPOTHESIS CONFIRMED: k=1 inner codes perform significantly better!")
    else:
        print("✗ HYPOTHESIS REJECTED: k=1 does not significantly outperform k=2")

HYPOTHESIS: k=1 inner codes should match BPOSD, k=2 should be worse
(v2: with string-format logical operator fix)

Testing: FourQubit422 [[4,2,2]] k=2
Inner code: n=4, k=2, d=2
Concatenated code: [[36, 10]]
  BPOSD:  LER = 0.0115
  SoftMP: LER = 0.1840
  Ratio (SoftMP/BPOSD): 16.0x

Testing: RepetitionCode [[3,1,3]] k=1
Inner code: n=3, k=1, d=3
Concatenated code: [[27, 1]]
  BPOSD:  LER = 0.0010
  SoftMP: LER = 0.1215
  Ratio (SoftMP/BPOSD): 121.5x

Testing: SteaneCode [[7,1,3]] k=1
Inner code: n=7, k=1, d=3
Concatenated code: [[63, 1]]
  BPOSD:  LER = 0.0055
  SoftMP: LER = 0.2525
  Ratio (SoftMP/BPOSD): 45.9x

SUMMARY: Does k=1 perform better than k=2?
Inner Code                            k    BPOSD   SoftMP    Ratio
----------------------------------------------------------------------
FourQubit422 [[4,2,2]] k=2            2   0.0115   0.1840    16.0x
RepetitionCode [[3,1,3]] k=1          1   0.0010   0.1215   121.5x
SteaneCode [[7,1,3]] k=1               1   0.0055   0.2525    45

In [ ]:
"""Investigate the logical operators for all test codes"""
print("=" * 70)
print("LOGICAL OPERATOR STRUCTURE FOR INNER CODES")
print("=" * 70)

for name, inner in inner_codes.items():
    print(f"\n{name}")
    print("-" * 50)
    print(f"  n={inner.n}, k={inner.k}")
    
    # Check logical_z_ops attribute (dict format)
    if hasattr(inner, 'logical_z_ops'):
        print(f"  logical_z_ops: {inner.logical_z_ops}")
    else:
        print(f"  logical_z_ops: NOT FOUND")
    
    # Check logical_x_ops attribute (dict format)
    if hasattr(inner, 'logical_x_ops'):
        print(f"  logical_x_ops: {inner.logical_x_ops}")
    else:
        print(f"  logical_x_ops: NOT FOUND")
    
    # Check logical_z (matrix form)
    if hasattr(inner, 'logical_z') and inner.logical_z is not None:
        lz = np.array(inner.logical_z)
        print(f"  logical_z (matrix): shape={lz.shape}")
        print(f"    {lz}")
    else:
        print(f"  logical_z (matrix): NOT FOUND")
    
    # Check logical_x (matrix form)
    if hasattr(inner, 'logical_x') and inner.logical_x is not None:
        lx = np.array(inner.logical_x)
        print(f"  logical_x (matrix): shape={lx.shape}")
        print(f"    {lx}")
    else:
        print(f"  logical_x (matrix): NOT FOUND")
    
    # Check metadata hints
    if hasattr(inner, 'metadata'):
        if 'logical_z_support' in inner.metadata:
            print(f"  metadata['logical_z_support']: {inner.metadata['logical_z_support']}")
        if 'logical_x_support' in inner.metadata:
            print(f"  metadata['logical_x_support']: {inner.metadata['logical_x_support']}")

print("\n" + "=" * 70)
print("CONCLUSION: Which logical format does each code use?")
print("=" * 70)

LOGICAL OPERATOR STRUCTURE FOR INNER CODES

FourQubit422 [[4,2,2]] k=2
--------------------------------------------------
  n=4, k=2
  logical_z_ops: [{0: 'Z', 2: 'Z'}, {1: 'Z', 3: 'Z'}]
  logical_x_ops: [{0: 'X', 1: 'X'}, {1: 'X', 2: 'X'}]
  logical_z (matrix): NOT FOUND
  logical_x (matrix): NOT FOUND
  metadata['logical_z_support']: [0, 2]
  metadata['logical_x_support']: [0, 1]

RepetitionCode [[3,1,3]] k=1
--------------------------------------------------
  n=3, k=1
  logical_z_ops: ['ZII']
  logical_x_ops: ['XXX']
  logical_z (matrix): NOT FOUND
  logical_x (matrix): NOT FOUND

SteaneCode [[7,1,3]] k=1
--------------------------------------------------
  n=7, k=1
  logical_z_ops: ['ZZZIIII']
  logical_x_ops: ['XXXIIII']
  logical_z (matrix): NOT FOUND
  logical_x (matrix): NOT FOUND
  metadata['logical_z_support']: [0, 1, 2]
  metadata['logical_x_support']: [0, 1, 2]

CONCLUSION: Which logical format does each code use?


In [ ]:
"""
DEEP DIVE: Compare detector structure for k=1 vs k=2 codes
=========================================================
Let's see if the detector slicing is correct for each code type.
"""
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.decoders.block_extraction import BlockExtractor
from qectostim.noise import CircuitDepolarizingNoise

print("=" * 70)
print("DETECTOR STRUCTURE COMPARISON")
print("=" * 70)

P_ERR = 0.01
ROUNDS = 1

for name, inner in inner_codes.items():
    print(f"\n{'='*70}")
    print(f"{name}")
    print("=" * 70)
    
    concat = ConcatenatedTopologicalCSSCode(outer, inner)
    
    # Build circuit and DEM
    try:
        exp = CSSMemoryExperiment(concat, rounds=ROUNDS)
        base_circuit = exp.to_stim()
        noise = CircuitDepolarizingNoise(P_ERR)
        circuit = noise.apply(base_circuit)
        # Allow decomposition failures for non-graph-like codes
        dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
    except Exception as e:
        print(f"  Circuit/DEM build FAILED: {e}")
        continue
    
    print(f"Code: [[{concat.n}, {concat.k}]]")
    print(f"Circuit: {circuit.num_qubits} qubits, {circuit.num_detectors} detectors, {circuit.num_observables} observables")
    print(f"DEM: {dem.num_detectors} detectors, {dem.num_errors} errors")
    
    # Try to build BlockExtractor
    try:
        extractor = BlockExtractor(concat, dem, rounds=ROUNDS)
        print(f"\nBlockExtractor:")
        print(f"  n_blocks: {extractor.n_blocks}")
        print(f"  n_inner_qubits: {extractor.n_inner_qubits}")
        print(f"  n_inner_x_checks: {extractor.n_inner_x_checks}")
        print(f"  n_inner_z_checks: {extractor.n_inner_z_checks}")
        print(f"  dets_per_block_per_round: {extractor.n_inner_x_checks + extractor.n_inner_z_checks}")
        
        # Check total detector accounting
        expected_inner_total = extractor.n_blocks * (extractor.n_inner_x_checks + extractor.n_inner_z_checks) * ROUNDS
        print(f"\n  Expected inner detectors: {expected_inner_total}")
        print(f"  Total DEM detectors: {dem.num_detectors}")
        print(f"  Outer detectors: ~{dem.num_detectors - expected_inner_total}")
        
        # Check first few block slices
        print(f"\n  Sample block slices:")
        for i in range(min(3, extractor.n_blocks)):
            start, stop = extractor.inner_slices[i]
            print(f"    Block {i}: detectors [{start}, {stop}) = {stop-start} detectors")
        
    except Exception as e:
        print(f"  BlockExtractor FAILED: {e}")
        import traceback
        traceback.print_exc()

DETECTOR STRUCTURE COMPARISON

FourQubit422 [[4,2,2]] k=2
Code: [[36, 10]]
Circuit: 62 qubits, 26 detectors, 1 observables
DEM: 26 detectors, 31 errors

BlockExtractor:
  n_blocks: 9
  n_inner_qubits: 4
  n_inner_x_checks: 1
  n_inner_z_checks: 1
  dets_per_block_per_round: 2

  Expected inner detectors: 18
  Total DEM detectors: 26
  Outer detectors: ~8

  Sample block slices:
    Block 0: detectors [0, 2) = 2 detectors
    Block 1: detectors [2, 4) = 2 detectors
    Block 2: detectors [4, 6) = 2 detectors

RepetitionCode [[3,1,3]] k=1
Code: [[27, 1]]
Circuit: 53 qubits, 44 detectors, 1 observables
DEM: 44 detectors, 49 errors

BlockExtractor:
  n_blocks: 9
  n_inner_qubits: 3
  n_inner_x_checks: 0
  n_inner_z_checks: 2
  dets_per_block_per_round: 2

  Expected inner detectors: 18
  Total DEM detectors: 44
  Outer detectors: ~26

  Sample block slices:
    Block 0: detectors [0, 4) = 4 detectors
    Block 1: detectors [4, 8) = 4 detectors
    Block 2: detectors [8, 12) = 4 detectors



In [ ]:
"""
DIAGNOSE: Trace through SoftMP decoding to see where it fails
============================================================
Focus on FourQubit422 since that has correct block slices
"""
import numpy as np

# Use FourQubit422 which has correct-looking slices
inner = FourQubit422Code()
concat = ConcatenatedTopologicalCSSCode(outer, inner)

exp = CSSMemoryExperiment(concat, rounds=1)
base_circuit = exp.to_stim()
noise = CircuitDepolarizingNoise(0.01)
circuit = noise.apply(base_circuit)
dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

print("=" * 70)
print("TRACE: SoftMP decoding step by step")
print("=" * 70)

# Sample some noisy syndromes
sampler = circuit.compile_detector_sampler()
n_samples = 100
dets, obs = sampler.sample(shots=n_samples, separate_observables=True)

# Initialize decoder
softmp = SoftMessagePassingDecoder(concat, dem)

print(f"Code: [[{concat.n}, {concat.k}]]")
print(f"Detector matrix shape: {dets.shape}")
print(f"Observable shape: {obs.shape}")
print(f"\nSoftMP config:")
print(f"  n_blocks: {softmp.n_blocks}")
print(f"  inner_slices: {list(softmp.inner_slices.items())[:3]}...")
print(f"  outer_slice: {softmp.outer_slice}")
print(f"  inner_lz: {softmp.inner_lz}")
print(f"  outer_L: {softmp.outer_L}")

# Trace through a few samples with syndrome triggered
n_trace = 0
errors = 0
softmp_predictions = []
for i in range(n_samples):
    syndrome = dets[i]
    actual_obs = obs[i][0]
    
    if np.sum(syndrome) > 0 and n_trace < 5:
        n_trace += 1
        print(f"\n--- Sample {i} (has syndrome) ---")
        print(f"Syndrome triggered: {np.sum(syndrome)} detectors")
        print(f"Actual observable: {actual_obs}")
        
        # Decode step by step manually
        inner_hard = np.zeros(softmp.n_blocks, dtype=np.uint8)
        inner_probs = np.zeros(softmp.n_blocks, dtype=np.float64)
        
        for block_id in range(softmp.n_blocks):
            det_start, det_stop = softmp.inner_slices[block_id]
            block_syn = syndrome[det_start:det_stop]
            hard, p_log = softmp._compute_inner_logical_prob(block_id, block_syn)
            inner_hard[block_id] = hard
            inner_probs[block_id] = p_log
        
        print(f"Inner hard decisions: {inner_hard}")
        print(f"Inner logical probs: {np.round(inner_probs, 3)}")
        
    # Always collect predictions for statistics
    pred = softmp.decode(syndrome)
    softmp_predictions.append(pred[0])
    if pred[0] != actual_obs:
        errors += 1

softmp_ler = errors / n_samples
print(f"\n" + "=" * 70)
print(f"SoftMP Errors: {errors}/{n_samples} = {softmp_ler:.1%}")

# Compare to BPOSD
bposd = BPOSDDecoder(dem)
bposd_errors = 0
for i in range(n_samples):
    syndrome = dets[i]
    actual_obs = obs[i][0]
    pred = bposd.decode(syndrome)
    if pred[0] != actual_obs:
        bposd_errors += 1

bposd_ler = bposd_errors / n_samples
print(f"BPOSD Errors: {bposd_errors}/{n_samples} = {bposd_ler:.1%}")
print(f"Ratio: {softmp_ler/bposd_ler if bposd_ler > 0 else 'inf'}x")
print("=" * 70)

TRACE: SoftMP decoding step by step
Code: [[36, 10]]
Detector matrix shape: (100, 26)
Observable shape: (100, 1)

SoftMP config:
  n_blocks: 9
  inner_slices: [(0, (0, 6)), (1, (6, 12)), (2, (12, 18))]...
  outer_slice: (54, 26)
  inner_lz: [1 0 1 0]
  outer_L: [1 1 1 0 0 0 0 0 0]

--- Sample 0 (has syndrome) ---
Syndrome triggered: 3 detectors
Actual observable: True
Inner hard decisions: [0 0 0 0 0 0 0 0 0]
Inner logical probs: [0. 0. 0. 0. 0. 0. 0. 0. 0.]

--- Sample 1 (has syndrome) ---
Syndrome triggered: 3 detectors
Actual observable: False
Inner hard decisions: [0 0 0 0 0 0 0 0 0]
Inner logical probs: [0.    0.    0.    0.375 0.    0.    0.    0.    0.   ]

--- Sample 4 (has syndrome) ---
Syndrome triggered: 4 detectors
Actual observable: True
Inner hard decisions: [0 0 0 0 0 0 0 0 0]
Inner logical probs: [0.    0.    0.375 0.    0.    0.    0.    0.    0.   ]

--- Sample 6 (has syndrome) ---
Syndrome triggered: 2 detectors
Actual observable: False
Inner hard decisions: [0 0 0 0

AttributeError: 'BPOSDDecoder' object has no attribute 'decode'

In [ ]:
"""
TRACE: Where is the 6-detector slice coming from?
=================================================
"""
print("=" * 70)
print("DEBUGGING: Detector slice computation")
print("=" * 70)

# This is from the previous diagnostic
inner = FourQubit422Code()
concat = ConcatenatedTopologicalCSSCode(outer, inner)

exp = CSSMemoryExperiment(concat, rounds=1)  # Explicit rounds=1
base_circuit = exp.to_stim()
noise = CircuitDepolarizingNoise(0.01)
circuit = noise.apply(base_circuit)
dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

print(f"Circuit rounds: 1 (explicit)")
print(f"Circuit detectors: {circuit.num_detectors}")
print(f"DEM detectors: {dem.num_detectors}")

# Create SoftMP with explicit rounds=1
print("\nCreating SoftMP with rounds=1:")
softmp = SoftMessagePassingDecoder(concat, dem, rounds=1)

print(f"  softmp.rounds: {softmp.rounds}")
print(f"  softmp.n_inner_x_checks: {softmp.n_inner_x_checks}")
print(f"  softmp.n_inner_z_checks: {softmp.n_inner_z_checks}")
print(f"  dets_per_block_per_round: {softmp.n_inner_x_checks + softmp.n_inner_z_checks}")
print(f"  Expected dets_per_block: {(softmp.n_inner_x_checks + softmp.n_inner_z_checks) * softmp.rounds}")
print(f"  softmp.dets_per_block: {softmp.dets_per_block}")
print(f"  softmp.inner_slices (first 3): {list(softmp.inner_slices.items())[:3]}")
print(f"  softmp.outer_slice: {softmp.outer_slice}")
print(f"  softmp.num_detectors: {softmp.num_detectors}")

# Now check what SHOULD be happening
print("\nExpected distribution:")
expected_inner = 9 * softmp.dets_per_block
print(f"  9 blocks × {softmp.dets_per_block} dets/block = {expected_inner} inner detectors")
print(f"  Remaining for outer: {dem.num_detectors - expected_inner}")

# Check if outer_slice makes sense
outer_start, outer_stop = softmp.outer_slice
print(f"\nOuter slice: ({outer_start}, {outer_stop})")
print(f"  This is {outer_stop - outer_start} detectors")
print(f"  But DEM has {dem.num_detectors} total detectors")

if outer_start > dem.num_detectors:
    print(f"  ⚠️ ERROR: outer_start ({outer_start}) > num_detectors ({dem.num_detectors})!")
    print(f"  This means inner slices EXCEED the actual detector count!")

DEBUGGING: Detector slice computation
Circuit rounds: 1 (explicit)
Circuit detectors: 26
DEM detectors: 26

Creating SoftMP with rounds=1:
  softmp.rounds: 1
  softmp.n_inner_x_checks: 1
  softmp.n_inner_z_checks: 1
  dets_per_block_per_round: 2
  Expected dets_per_block: 2
  softmp.dets_per_block: 2
  softmp.inner_slices (first 3): [(0, (0, 2)), (1, (2, 4)), (2, (4, 6))]
  softmp.outer_slice: (18, 26)
  softmp.num_detectors: 26

Expected distribution:
  9 blocks × 2 dets/block = 18 inner detectors
  Remaining for outer: 8

Outer slice: (18, 26)
  This is 8 detectors
  But DEM has 26 total detectors


In [ ]:
"""
PROPERLY CONFIGURED COMPARISON
==============================
Now with correct rounds=1 configuration throughout.
"""
import numpy as np

print("=" * 70)
print("PROPERLY CONFIGURED DECODER COMPARISON")
print("=" * 70)

# Use FourQubit422 for this analysis
inner = FourQubit422Code()
concat = ConcatenatedTopologicalCSSCode(outer, inner)

# Build circuit with rounds=1
ROUNDS = 1
exp = CSSMemoryExperiment(concat, rounds=ROUNDS)
base_circuit = exp.to_stim()
noise = CircuitDepolarizingNoise(0.01)
circuit = noise.apply(base_circuit)
dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

print(f"Code: [[{concat.n}, {concat.k}]]")
print(f"Circuit: {circuit.num_detectors} detectors, rounds={ROUNDS}")

# Create decoders with CORRECT rounds
softmp = SoftMessagePassingDecoder(concat, dem, rounds=ROUNDS)
bposd = BPOSDDecoder(dem)

print(f"\nSoftMP detector config:")
print(f"  n_blocks: {softmp.n_blocks}")
print(f"  dets_per_block: {softmp.dets_per_block}")
print(f"  inner total: {softmp.n_blocks * softmp.dets_per_block}")
print(f"  outer slice: {softmp.outer_slice}")
print(f"  inner_lz: {softmp.inner_lz}")
print(f"  outer_L: {softmp.outer_L}")

# Sample and decode
sampler = circuit.compile_detector_sampler()
SHOTS = 500
dets, obs = sampler.sample(shots=SHOTS, separate_observables=True)

softmp_errors = 0
bposd_errors = 0

# Detailed analysis of failures
softmp_failures = []
bposd_failures = []

for i in range(SHOTS):
    syndrome = dets[i]
    actual = obs[i][0]
    
    softmp_pred = softmp.decode(syndrome)[0]
    bposd_pred = bposd.decode_batch(syndrome.reshape(1, -1))[0, 0]
    
    if softmp_pred != actual:
        softmp_errors += 1
        if len(softmp_failures) < 5:
            softmp_failures.append({
                'idx': i,
                'syn_weight': np.sum(syndrome),
                'actual': actual,
                'pred': softmp_pred,
                'syndrome': syndrome.copy()
            })
    
    if bposd_pred != actual:
        bposd_errors += 1
        if len(bposd_failures) < 5:
            bposd_failures.append({
                'idx': i,
                'syn_weight': np.sum(syndrome),
                'actual': actual,
                'pred': bposd_pred,
            })

softmp_ler = softmp_errors / SHOTS
bposd_ler = bposd_errors / SHOTS

print(f"\n{'='*70}")
print(f"Results at p=0.01, rounds=1, shots={SHOTS}")
print(f"{'='*70}")
print(f"BPOSD:  LER = {bposd_ler:.4f} ({bposd_errors}/{SHOTS} errors)")
print(f"SoftMP: LER = {softmp_ler:.4f} ({softmp_errors}/{SHOTS} errors)")
print(f"Ratio: {softmp_ler/bposd_ler:.1f}x" if bposd_ler > 0 else "N/A")

# Analyze SoftMP failures
print(f"\n{'='*70}")
print("SoftMP failure analysis (first 5):")
print("{'='*70}")
for f in softmp_failures:
    print(f"\nSample {f['idx']}:")
    print(f"  Syndrome weight: {f['syn_weight']}")
    print(f"  Actual: {f['actual']}, Predicted: {f['pred']}")
    
    # Trace through the decoding
    syn = f['syndrome']
    inner_hard = np.zeros(softmp.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(softmp.n_blocks, dtype=np.float64)
    
    for block_id in range(softmp.n_blocks):
        det_start, det_stop = softmp.inner_slices[block_id]
        block_syn = syn[det_start:det_stop]
        hard, p_log = softmp._compute_inner_logical_prob(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    print(f"  Inner hard: {inner_hard}")
    print(f"  Inner probs: {np.round(inner_probs, 3)}")

PROPERLY CONFIGURED DECODER COMPARISON
Code: [[36, 10]]
Circuit: 26 detectors, rounds=1

SoftMP detector config:
  n_blocks: 9
  dets_per_block: 2
  inner total: 18
  outer slice: (18, 26)
  inner_lz: [1 0 1 0]
  outer_L: [1 1 1 0 0 0 0 0 0]

Results at p=0.01, rounds=1, shots=500
BPOSD:  LER = 0.0000 (0/500 errors)
SoftMP: LER = 0.0560 (28/500 errors)
N/A

SoftMP failure analysis (first 5):
{'='*70}

Sample 3:
  Syndrome weight: 2
  Actual: True, Predicted: 0
  Inner hard: [0 0 0 0 0 0 0 0 0]
  Inner probs: [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]

Sample 13:
  Syndrome weight: 1
  Actual: False, Predicted: 1
  Inner hard: [0 0 0 0 0 0 0 0 0]
  Inner probs: [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002]

Sample 23:
  Syndrome weight: 3
  Actual: True, Predicted: 0
  Inner hard: [0 0 0 0 0 0 0 0 0]
  Inner probs: [0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.375 0.375]

Sample 54:
  Syndrome weight: 1
  Actual: False, Predicted: 1
  Inner hard: [0 0 0 0 0 0 0 0 0

In [ ]:
"""
DEBUG: What do the block syndromes actually represent?
=====================================================
"""
print("=" * 70)
print("SYNDROME STRUCTURE ANALYSIS")
print("=" * 70)

# Look at what detectors correspond to for FourQubit422
print("\nInner code (FourQubit422):")
print(f"  hx (detects Z errors): {inner.hx}")
print(f"  hz (detects X errors): {inner.hz}")
print(f"  n_inner_x_checks: {softmp.n_inner_x_checks}")
print(f"  n_inner_z_checks: {softmp.n_inner_z_checks}")

print(f"\nCircuit detectors: {circuit.num_detectors}")

# Sample a few syndromes and see the patterns
print("\nSample syndrome patterns:")
for i in range(min(10, SHOTS)):
    syn = dets[i]
    if np.sum(syn) > 0:
        print(f"\nSample {i}: total triggered = {np.sum(syn)}")
        print(f"  Full syndrome: {syn}")
        
        # Look at each block
        for b in range(softmp.n_blocks):
            start, stop = softmp.inner_slices[b]
            block_syn = syn[start:stop]
            if np.sum(block_syn) > 0:
                print(f"  Block {b}: {block_syn} (dets {start}:{stop})")
        
        # Outer detectors
        outer_start, outer_stop = softmp.outer_slice
        outer_syn = syn[outer_start:outer_stop]
        if np.sum(outer_syn) > 0:
            print(f"  Outer: {outer_syn} (dets {outer_start}:{outer_stop})")

# Check if detectors are actually firing in blocks
print("\n" + "=" * 70)
print("Block detector statistics:")
print("=" * 70)

block_activity = np.zeros(softmp.n_blocks)
for i in range(SHOTS):
    for b in range(softmp.n_blocks):
        start, stop = softmp.inner_slices[b]
        block_activity[b] += np.any(dets[i, start:stop])

print("Blocks with any syndrome activity (out of {} samples):".format(SHOTS))
for b, count in enumerate(block_activity):
    print(f"  Block {b}: {int(count)} samples ({count/SHOTS*100:.1f}%)")

SYNDROME STRUCTURE ANALYSIS

Inner code (FourQubit422):
  hx (detects Z errors): [[1 1 1 1]]
  hz (detects X errors): [[1 1 1 1]]
  n_inner_x_checks: 1
  n_inner_z_checks: 1

Circuit detectors: 26

Sample syndrome patterns:

Sample 1: total triggered = 2
  Full syndrome: [False False False False False False False  True False False False False
 False False False False False False False False  True False False False
 False False]
  Block 3: [False  True] (dets 6:8)
  Outer: [False False  True False False False False False] (dets 18:26)

Sample 3: total triggered = 2
  Full syndrome: [False False False False False False False False False False False False
 False  True False False False False False False False False False  True
 False False]
  Block 6: [False  True] (dets 12:14)
  Outer: [False False False False False  True False False] (dets 18:26)

Sample 4: total triggered = 3
  Full syndrome: [False False False False False False False False False False False False
 False False False Fa

In [ ]:
"""
KEY INSIGHT: The detector layout doesn't match SoftMP's assumptions!
====================================================================
CSSMemoryExperiment lays out detectors differently than a concatenated structure.
Let's verify this by looking at the circuit.
"""
print("=" * 70)
print("CIRCUIT STRUCTURE ANALYSIS")
print("=" * 70)

# Print a summary of the circuit
print(f"\nCircuit summary:")
print(f"  Qubits: {circuit.num_qubits}")
print(f"  Detectors: {circuit.num_detectors}")
print(f"  Observables: {circuit.num_observables}")

# Look at detector coords to understand layout
print("\nDetector coordinates (if available):")
det_coords = {}
for line in str(circuit).split('\n'):
    if line.startswith('DETECTOR'):
        # Parse detector index and coords
        parts = line.split()
        if len(parts) >= 2:
            try:
                coords = [float(x) for x in parts[1].strip('()').split(',')]
                det_idx = len(det_coords)
                det_coords[det_idx] = coords
            except:
                pass

# Print first 20 detector coords
for det_idx in range(min(26, len(det_coords))):
    if det_idx in det_coords:
        coords = det_coords[det_idx]
        print(f"  D{det_idx}: {coords}")

# Check if detectors are grouped by block
print(f"\nSoftMP expects:")
print(f"  {softmp.n_blocks} blocks × {softmp.dets_per_block} dets = {softmp.n_blocks * softmp.dets_per_block} inner")
print(f"  {softmp.outer_slice[1] - softmp.outer_slice[0]} outer detectors")
print(f"  Total: {circuit.num_detectors}")

print(f"\nBut the circuit may have a different layout!")
print(f"CSSMemoryExperiment uses the concatenated code's flat stabilizer structure,")
print(f"not a block-by-block inner code structure.")

CIRCUIT STRUCTURE ANALYSIS

Circuit summary:
  Qubits: 62
  Detectors: 26
  Observables: 1

Detector coordinates (if available):

SoftMP expects:
  9 blocks × 2 dets = 18 inner
  8 outer detectors
  Total: 26

But the circuit may have a different layout!
CSSMemoryExperiment uses the concatenated code's flat stabilizer structure,
not a block-by-block inner code structure.


In [ ]:
"""
TRY: ConcatenatedMemoryExperiment for block-grouped detectors
=============================================================
"""
from qectostim.experiments.concatenated_memory import ConcatenatedMemoryExperiment

print("=" * 70)
print("USING ConcatenatedMemoryExperiment")
print("=" * 70)

try:
    # Build circuit with ConcatenatedMemoryExperiment
    concat_exp = ConcatenatedMemoryExperiment(concat, rounds=1, basis="Z")
    concat_circuit = noise.apply(concat_exp.to_stim())
    concat_dem = concat_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
    
    print(f"ConcatenatedMemoryExperiment circuit:")
    print(f"  Qubits: {concat_circuit.num_qubits}")
    print(f"  Detectors: {concat_circuit.num_detectors}")
    print(f"  Observables: {concat_circuit.num_observables}")
    
    # Compare to CSSMemoryExperiment
    print(f"\nCSSMemoryExperiment circuit:")
    print(f"  Qubits: {circuit.num_qubits}")
    print(f"  Detectors: {circuit.num_detectors}")
    print(f"  Observables: {circuit.num_observables}")
    
    if concat_circuit.num_detectors != circuit.num_detectors:
        print(f"\n⚠️ Different detector counts! ConcatExp has {concat_circuit.num_detectors}, CSSExp has {circuit.num_detectors}")
    
    # Test SoftMP with ConcatenatedMemoryExperiment circuit
    softmp_concat = SoftMessagePassingDecoder(concat, concat_dem, rounds=1)
    
    print(f"\nSoftMP with ConcatenatedMemoryExperiment:")
    print(f"  n_blocks: {softmp_concat.n_blocks}")
    print(f"  dets_per_block: {softmp_concat.dets_per_block}")
    print(f"  inner_slices: {list(softmp_concat.inner_slices.items())[:3]}...")
    print(f"  outer_slice: {softmp_concat.outer_slice}")
    
    # Sample and compare decoders
    concat_sampler = concat_circuit.compile_detector_sampler()
    concat_dets, concat_obs = concat_sampler.sample(shots=200, separate_observables=True)
    
    softmp_errors = 0
    bposd_concat = BPOSDDecoder(concat_dem)
    bposd_errors = 0
    
    for i in range(200):
        syn = concat_dets[i]
        actual = concat_obs[i][0]
        
        softmp_pred = softmp_concat.decode(syn)[0]
        bposd_pred = bposd_concat.decode_batch(syn.reshape(1, -1))[0, 0]
        
        if softmp_pred != actual:
            softmp_errors += 1
        if bposd_pred != actual:
            bposd_errors += 1
    
    print(f"\n{'='*70}")
    print(f"Results with ConcatenatedMemoryExperiment (200 samples):")
    print(f"{'='*70}")
    print(f"BPOSD:  {bposd_errors}/200 = {bposd_errors/200:.1%}")
    print(f"SoftMP: {softmp_errors}/200 = {softmp_errors/200:.1%}")
    
except Exception as e:
    print(f"ConcatenatedMemoryExperiment failed: {e}")
    import traceback
    traceback.print_exc()

USING ConcatenatedMemoryExperiment
ConcatenatedMemoryExperiment failed: The circuit contains non-deterministic observables.

To make an SVG picture of the problem, you can use the python API like this:
    your_circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['L8', 'L9', ])
or the command line API like this:
    stim diagram --in your_circuit_file.stim --type detslice-with-ops-svg --tick 0:5 --filter_coords L8:L9 > output_image.svg

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 48

The collapse anti-commuted with these detectors/observables:
    L8
    L9

The backward-propagating error sensitivity for L8 was:
    Z28 [coords (-0.5, 3.74264)]
    Z30 [coords (0.5, 4.74264)]
    X46
    X48

The backward-propagating error sensitivity for L9 was:
    Z32 [coords (3.74264, 3.74264)]
    Z34 [coords (4.74264, 4.74264)]
    X48

Circuit stack trace:
    during TICK layer #1 of 19
    at instruction #37 [which is R 0 1 2 3 4 5 6 7 8 9 10 11 12

Traceback (most recent call last):
  File "/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_39840/455100180.py", line 15, in <module>
    concat_dem = concat_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: The circuit contains non-deterministic observables.

To make an SVG picture of the problem, you can use the python API like this:
    your_circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['L8', 'L9', ])
or the command line API like this:
    stim diagram --in your_circuit_file.stim --type detslice-with-ops-svg --tick 0:5 --filter_coords L8:L9 > output_image.svg

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 48

The collapse anti-commuted with these detectors/observables:
    L8
    L9

The backward-propagating error sensitivity for L8 was:
    Z28 [coords (-0.5, 3.

In [4]:
# Debug: Compare Template DEM vs actual inner block detector counts
import numpy as np
from qectostim.codes.small.steane_713 import SteaneCode713
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.experiments.concatenated_memory import ConcatenatedMemoryExperiment
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

# Build concatenated code
inner = SteaneCode713()
outer = RotatedSurfaceCode(distance=3)
concat = ConcatenatedTopologicalCSSCode(outer=outer, inner=inner)

P_ERR = 0.005
ROUNDS = 1

print("="*70)
print("DEBUGGING: Template DEM vs Actual Inner Block Detector Structure")
print("="*70)

print(f"\nInner code: {inner.name}")
print(f"  n={inner.n}, k={inner.k}")
print(f"  hx shape: {inner.hx.shape}, hz shape: {inner.hz.shape}")
print(f"  X stabilizers: {inner.hx.shape[0]}, Z stabilizers: {inner.hz.shape[0]}")

print(f"\nOuter code: {outer.name}")
print(f"  n={outer.n}, k={outer.k}")

# Build concatenated circuit
noise = CircuitDepolarizingNoise(p1=P_ERR, p2=P_ERR, before_measure_flip=P_ERR)
concat_exp = ConcatenatedMemoryExperiment(concat, noise_model=P_ERR, rounds=ROUNDS)
concat_circuit = concat_exp.to_stim()
concat_dem = concat_circuit.detector_error_model(decompose_errors=True)

print(f"\n--- Concatenated Circuit ---")
print(f"Total detectors: {concat_circuit.num_detectors}")
print(f"Total observables: {concat_circuit.num_observables}")

# Get detector slices from experiment
slices = concat_exp.get_detector_slices()
print(f"\nDetector slices from experiment:")
for key, val in slices.items():
    print(f"  {key}: {val}")

# Build Template DEM for inner code
print(f"\n--- Template DEM for Inner Code ---")
template_exp = CSSMemoryExperiment(
    code=inner,
    rounds=ROUNDS,
    noise_model=noise,
    basis='Z'
)
template_circuit = template_exp.to_stim()
noisy_template = noise.apply(template_circuit)
template_dem = noisy_template.detector_error_model(decompose_errors=True)

print(f"Template circuit detectors: {template_circuit.num_detectors}")
print(f"Template circuit observables: {template_circuit.num_observables}")
print(f"Template DEM detectors: {template_dem.num_detectors}")
print(f"Template DEM observables: {template_dem.num_observables}")

# Check inner block size from concatenated circuit
if 'inner_slices' in slices:
    inner_slices = slices['inner_slices']
    if len(inner_slices) > 0:
        first_block_start, first_block_end = inner_slices[0]
        actual_inner_dets = first_block_end - first_block_start
        print(f"\nActual inner block 0 detector count: {actual_inner_dets}")
        print(f"Template DEM detector count: {template_dem.num_detectors}")
        print(f"MATCH: {actual_inner_dets == template_dem.num_detectors}")

DEBUGGING: Template DEM vs Actual Inner Block Detector Structure

Inner code: SteaneCode713
  n=7, k=1
  hx shape: (3, 7), hz shape: (3, 7)
  X stabilizers: 3, Z stabilizers: 3

Outer code: RotatedSurfaceCode
  n=9, k=1

--- Concatenated Circuit ---
Total detectors: 62
Total observables: 1

Detector slices from experiment:
  inner_slices: {0: (0, 6), 1: (6, 12), 2: (12, 18), 3: (18, 24), 4: (24, 30), 5: (30, 36), 6: (36, 42), 7: (42, 48), 8: (48, 54)}
  outer_slices: {0: (54, 62)}
  inner_dets_per_block: 6
  outer_dets: 8
  n_inner_blocks: 9
  total_detectors: 62
  rounds: 1
  basis: Z

--- Template DEM for Inner Code ---
Template circuit detectors: 6
Template circuit observables: 1
Template DEM detectors: 6
Template DEM observables: 1

Actual inner block 0 detector count: 6
Template DEM detector count: 6
MATCH: True


In [5]:
# Debug: Test SoftHierarchical initialization directly
import warnings
from qectostim.decoders.soft_hierarchical_decoder import SoftHierarchicalDecoder

print("="*70)
print("DEBUGGING: SoftHierarchical Initialization")
print("="*70)

# Capture all warnings during initialization
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    
    soft_hier = SoftHierarchicalDecoder(
        code=concat,
        dem=concat_dem,
        rounds=ROUNDS,
        basis='Z'
    )
    
    print("\nWarnings during initialization:")
    for warning in w:
        print(f"  - {warning.message}")

print(f"\n--- SoftHierarchical State ---")
print(f"_inner_decoder_method: {soft_hier._inner_decoder_method}")
print(f"_has_inner_observables: {soft_hier._has_inner_observables}")
print(f"n_blocks: {soft_hier.n_blocks}")
print(f"inner_slices: {soft_hier.inner_slices}")
print(f"Number of inner_decoders: {len(soft_hier.inner_decoders)}")

# Check what's in inner_decoders
print(f"\nInner decoder details:")
for block_id in range(min(3, soft_hier.n_blocks)):
    decoder = soft_hier.inner_decoders.get(block_id)
    obs_mat = soft_hier.inner_obs_matrices.get(block_id)
    print(f"  Block {block_id}:")
    print(f"    decoder: {type(decoder).__name__ if decoder else None}")
    if obs_mat is not None:
        print(f"    obs_matrix shape: {obs_mat.shape}")
    else:
        print(f"    obs_matrix: None")

DEBUGGING: SoftHierarchical Initialization

Warnings during initialization:
  - SoftHierarchicalDecoder is deprecated. Use SoftMessagePassingDecoder instead for more robust hierarchical decoding with adaptive detector slicing.
  - SoftHierarchicalDecoder: DEM has 1 observables, expected 10 (1 main + 9 inner blocks). Using code-based soft information estimation (may be less accurate).
  - This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  - This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  - This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  - This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  - This is the old syntax for the `bposd_decoder` from `ldpc v1`. Us

In [7]:
# Debug: Test actual decoding on a sample shot
print("="*70)
print("DEBUGGING: Decode a single shot step by step")
print("="*70)

# Sample with higher error rate to get more errors
high_noise = CircuitDepolarizingNoise(p1=0.05, p2=0.05, before_measure_flip=0.05)
high_noise_exp = ConcatenatedMemoryExperiment(concat, noise_model=0.05, rounds=ROUNDS)
high_noise_circuit = high_noise_exp.to_stim()

sampler = high_noise_circuit.compile_detector_sampler()
det_sample, obs_sample = sampler.sample(shots=100, separate_observables=True)

# Find a shot with non-zero syndrome
shot_idx = None
for i in range(100):
    if det_sample[i].sum() > 0:
        shot_idx = i
        break

if shot_idx is None:
    print("No shots with errors found, using shot 0")
    shot_idx = 0

syndrome = det_sample[shot_idx].astype(np.uint8)
true_obs = obs_sample[shot_idx]

print(f"Shot {shot_idx}: syndrome weight = {syndrome.sum()}, true observable = {true_obs}")

# Find block with non-zero syndrome
for block_id in range(soft_hier.n_blocks):
    det_start, det_stop = soft_hier.inner_slices[block_id]
    block_syndrome = syndrome[det_start:det_stop]
    if block_syndrome.sum() > 0:
        break

print(f"\nBlock {block_id} syndrome: {block_syndrome} (weight={block_syndrome.sum()})")

# Get the decoder and observable matrix
decoder = soft_hier.inner_decoders[block_id]
obs_matrix = soft_hier.inner_obs_matrices[block_id]

print(f"Template obs_matrix shape: {obs_matrix.shape}")

# Decode
try:
    error_pattern = decoder.decode(block_syndrome)
    print(f"Error pattern shape: {error_pattern.shape}, weight: {error_pattern.sum()}")
    
    # Check observable
    if obs_matrix is not None:
        obs_dense = obs_matrix.toarray() if hasattr(obs_matrix, 'toarray') else np.asarray(obs_matrix)
        hard_pred = int(obs_dense @ error_pattern % 2)
        print(f"Observable matrix:\n{obs_dense}")
        print(f"Hard prediction: {hard_pred}")
    
    # Check LLRs
    if hasattr(decoder, 'log_prob_ratios'):
        llrs = decoder.log_prob_ratios
        print(f"\nLLRs shape: {llrs.shape if llrs is not None else None}")
        if llrs is not None and len(llrs) > 0:
            print(f"LLRs: min={llrs.min():.2f}, max={llrs.max():.2f}, mean={llrs.mean():.2f}")
            print(f"LLRs all zero: {np.all(llrs == 0)}")
            print(f"First 10 LLRs: {llrs[:10]}")
except Exception as e:
    print(f"Decode failed: {e}")
    import traceback
    traceback.print_exc()

DEBUGGING: Decode a single shot step by step
No shots with errors found, using shot 0
Shot 0: syndrome weight = 0, true observable = [False]

Block 8 syndrome: [0 0 0 0 0 0] (weight=0)
Template obs_matrix shape: (1, 22)
Error pattern shape: (22,), weight: 0
Observable matrix:
[[0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1]]
Hard prediction: 0

LLRs shape: (22,)
LLRs: min=0.00, max=0.00, mean=0.00
LLRs all zero: True
First 10 LLRs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/2689492938.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hard_pred = int(obs_dense @ error_pattern % 2)


In [9]:
# Debug: Find a shot with actual syndrome and decode it properly
print("="*70)
print("DEBUGGING: Find shot with actual syndrome")
print("="*70)

# Build circuit with HIGHER error rate for testing
P_HIGH = 0.05  # 5% error rate
high_exp = ConcatenatedMemoryExperiment(concat, noise_model=P_HIGH, rounds=ROUNDS)
high_circuit = high_exp.to_stim()
high_dem = high_circuit.detector_error_model(decompose_errors=True)

print(f"Using error rate p={P_HIGH}")
print(f"Circuit detectors: {high_circuit.num_detectors}")

# Also rebuild soft_hier with this DEM
soft_hier_high = SoftHierarchicalDecoder(
    code=concat,
    dem=high_dem,
    rounds=ROUNDS,
    basis='Z'
)
print(f"SoftHierarchical _inner_decoder_method: {soft_hier_high._inner_decoder_method}")

# Sample shots
sampler = high_circuit.compile_detector_sampler()
det_sample, obs_sample = sampler.sample(shots=1000, separate_observables=True)

# Count shots with syndrome
shots_with_syndrome = np.sum(det_sample.sum(axis=1) > 0)
print(f"\nShots with syndrome: {shots_with_syndrome}/1000")

# Find a shot with inner syndrome
test_shot_idx = None
test_block_id = None
for i in range(1000):
    for block_id in range(soft_hier_high.n_blocks):
        det_start, det_stop = soft_hier_high.inner_slices[block_id]
        if det_sample[i, det_start:det_stop].sum() > 0:
            test_shot_idx = i
            test_block_id = block_id
            break
    if test_shot_idx is not None:
        break

if test_shot_idx is not None:
    print(f"\nFound shot {test_shot_idx} with syndrome in block {test_block_id}")
    
    syndrome = det_sample[test_shot_idx].astype(np.uint8)
    det_start, det_stop = soft_hier_high.inner_slices[test_block_id]
    block_syndrome = syndrome[det_start:det_stop]
    
    print(f"Block {test_block_id} syndrome: {block_syndrome}")
    
    # Decode
    decoder = soft_hier_high.inner_decoders[test_block_id]
    obs_matrix = soft_hier_high.inner_obs_matrices[test_block_id]
    
    error_pattern = decoder.decode(block_syndrome)
    print(f"Error pattern sum: {error_pattern.sum()}")
    
    # Check LLRs
    llrs = decoder.log_prob_ratios
    print(f"\nLLRs all zero: {np.all(llrs == 0)}")
    if not np.all(llrs == 0):
        print(f"LLRs: min={llrs.min():.2f}, max={llrs.max():.2f}")
        
        # Compute soft info
        obs_dense = obs_matrix.toarray() if hasattr(obs_matrix, 'toarray') else np.asarray(obs_matrix)
        
        # Convert LLRs to probabilities
        # LLR = log(P(no error) / P(error)) => P(error) = 1 / (1 + exp(LLR))
        posteriors = 1.0 / (1.0 + np.exp(llrs))
        print(f"Posteriors: min={posteriors.min():.4f}, max={posteriors.max():.4f}")
        
        # Use soft XOR to compute P(logical error)
        logical_support = obs_dense.flatten()
        p_logical = 0.0
        for i in range(len(logical_support)):
            if logical_support[i]:
                p_i = posteriors[i]
                p_logical = p_logical * (1 - p_i) + p_i * (1 - p_logical)
        print(f"Soft P(logical error): {p_logical:.4f}")
    else:
        print("LLRs are all zero - BP didn't converge or no errors")
else:
    print("No shot with inner syndrome found!")

DEBUGGING: Find shot with actual syndrome
Using error rate p=0.05
Circuit detectors: 62
SoftHierarchical _inner_decoder_method: template_dem

Shots with syndrome: 0/1000
No shot with inner syndrome found!


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/2464530077.py:16: DeprecationWarning: SoftHierarchicalDecoder is deprecated. Use SoftMessagePassingDecoder instead for more robust hierarchical decoding with adaptive detector slicing.
  soft_hier_high = SoftHierarchicalDecoder(
/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/decoders/soft_hierarchical_decoder.py:190: UserWarning: SoftHierarchicalDecoder: DEM has 1 observables, expected 10 (1 main + 9 inner blocks). Using code-based soft information estimation (may be less accurate).
  warnings.warn(
/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/my_venv/lib/python3.11/site-packages/ldpc/_legacy_ldpc_v1/_legacy_bposd_decoder.py:45: UserWarning: This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  warnings.warn(


In [10]:
# Debug: Verify circuit is actually noisy
print("="*70)
print("DEBUGGING: Verify circuit has noise")  
print("="*70)

# Check DEM has errors
print(f"DEM num_errors: {high_dem.num_errors}")
print(f"DEM num_detectors: {high_dem.num_detectors}")

# Print first few DEM instructions
print(f"\nFirst 20 DEM instructions:")
dem_str = str(high_dem)
lines = dem_str.split('\n')[:20]
for line in lines:
    print(f"  {line}")

# Also check raw circuit has DEPOLARIZE
circuit_str = str(high_circuit)
if 'DEPOLARIZE' in circuit_str or 'X_ERROR' in circuit_str:
    print("\nCircuit contains error operations ✓")
else:
    print("\nCircuit does NOT contain error operations ✗")
    
# Try stim's noise injection directly
test_circuit = stim.Circuit()
test_circuit.append("H", [0])
test_circuit.append("DEPOLARIZE1", [0], 0.1)
test_circuit.append("M", [0])

test_sampler = test_circuit.compile_sampler()
test_samples = test_sampler.sample(100)
print(f"\nTest circuit: {test_samples.sum()}/100 shots had '1' measurement")

DEBUGGING: Verify circuit has noise
DEM num_errors: 0
DEM num_detectors: 62

First 20 DEM instructions:
  shift_detectors(0, 0, 1) 0
  detector(0, 0, 0) D0
  detector(0, 1, 0) D1
  detector(0, 2, 0) D2
  detector(0, 0, 1) D3
  detector(0, 1, 1) D4
  detector(0, 2, 1) D5
  detector(1, 0, 0) D6
  detector(1, 1, 0) D7
  detector(1, 2, 0) D8
  detector(1, 0, 1) D9
  detector(1, 1, 1) D10
  detector(1, 2, 1) D11
  detector(2, 0, 0) D12
  detector(2, 1, 0) D13
  detector(2, 2, 0) D14
  detector(2, 0, 1) D15
  detector(2, 1, 1) D16
  detector(2, 2, 1) D17
  detector(3, 0, 0) D18

Circuit does NOT contain error operations ✗


NameError: name 'stim' is not defined

In [12]:
# Debug: Build circuit WITH PROPER NOISE APPLICATION
import stim
print("="*70)
print("DEBUGGING: Proper noise application")  
print("="*70)

P_TEST = 0.05
noise = CircuitDepolarizingNoise(p1=P_TEST, p2=P_TEST, before_measure_flip=P_TEST)

# Build experiment  
test_exp = ConcatenatedMemoryExperiment(concat, noise_model=noise, rounds=ROUNDS)

# Get ideal circuit
ideal_circuit = test_exp.to_stim()
print(f"Ideal circuit: {ideal_circuit.num_detectors} detectors")
print(f"Ideal circuit has DEPOLARIZE: {'DEPOLARIZE' in str(ideal_circuit)}")

# Apply noise 
noisy_circuit = noise.apply(ideal_circuit)
print(f"\nNoisy circuit has DEPOLARIZE: {'DEPOLARIZE' in str(noisy_circuit)}")

# Build DEM (allow hyperedges for concatenated code)
noisy_dem = noisy_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
print(f"Noisy DEM: {noisy_dem.num_errors} errors, {noisy_dem.num_detectors} detectors")

# Sample with noise
sampler = noisy_circuit.compile_detector_sampler()
det_sample, obs_sample = sampler.sample(shots=100, separate_observables=True)

shots_with_syndrome = np.sum(det_sample.sum(axis=1) > 0)
print(f"\nShots with syndrome: {shots_with_syndrome}/100")

# Now test SoftHierarchical with this properly noisy circuit/DEM
print("\n" + "="*70)
print("Testing SoftHierarchical with properly noisy circuit")
print("="*70)

# Rebuild SoftHierarchical with the noisy DEM
soft_hier_noisy = SoftHierarchicalDecoder(
    code=concat,
    dem=noisy_dem,
    rounds=ROUNDS,
    basis='Z'
)
print(f"_inner_decoder_method: {soft_hier_noisy._inner_decoder_method}")

# Find shot with inner syndrome
test_shot_idx = None
test_block_id = None
for i in range(100):
    for block_id in range(soft_hier_noisy.n_blocks):
        det_start, det_stop = soft_hier_noisy.inner_slices[block_id]
        if det_sample[i, det_start:det_stop].sum() > 0:
            test_shot_idx = i
            test_block_id = block_id
            break
    if test_shot_idx is not None:
        break

if test_shot_idx is not None:
    print(f"\nFound shot {test_shot_idx} with syndrome in block {test_block_id}")
    
    syndrome = det_sample[test_shot_idx].astype(np.uint8)
    det_start, det_stop = soft_hier_noisy.inner_slices[test_block_id]
    block_syndrome = syndrome[det_start:det_stop]
    
    print(f"Block {test_block_id} syndrome: {block_syndrome}")
    
    # Decode the block
    decoder = soft_hier_noisy.inner_decoders[test_block_id]
    error_pattern = decoder.decode(block_syndrome)
    
    # Check LLRs
    llrs = decoder.log_prob_ratios
    print(f"LLRs all zero: {np.all(llrs == 0)}")
    if not np.all(llrs == 0):
        print(f"LLRs: min={llrs.min():.2f}, max={llrs.max():.2f}, mean={llrs.mean():.2f}")
else:
    print("No shot with inner syndrome found!")

DEBUGGING: Proper noise application
Ideal circuit: 62 detectors
Ideal circuit has DEPOLARIZE: False

Noisy circuit has DEPOLARIZE: True
Noisy DEM: 303 errors, 62 detectors

Shots with syndrome: 100/100

Testing SoftHierarchical with properly noisy circuit
_inner_decoder_method: template_dem

Found shot 0 with syndrome in block 0
Block 0 syndrome: [0 0 1 1 1 0]
LLRs all zero: False
LLRs: min=0.61, max=4.90, mean=2.46


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/465652225.py:39: DeprecationWarning: SoftHierarchicalDecoder is deprecated. Use SoftMessagePassingDecoder instead for more robust hierarchical decoding with adaptive detector slicing.
  soft_hier_noisy = SoftHierarchicalDecoder(


In [13]:
# Test full decode and compute LER
print("="*70)
print("TEST: SoftHierarchical full decode with proper noise")
print("="*70)

# Sample more shots
SHOTS = 1000
det_samples, obs_samples = sampler.sample(shots=SHOTS, separate_observables=True)

print(f"Samples: {SHOTS} shots")
print(f"Shots with syndrome: {np.sum(det_samples.sum(axis=1) > 0)}/{SHOTS}")

# Decode all shots
predictions = soft_hier_noisy.decode_batch(det_samples.astype(np.uint8))

# Compute LER
errors = np.sum(predictions.flatten() != obs_samples.flatten())
ler = errors / SHOTS

print(f"\nSoftHierarchical LER: {ler:.4f}")
print(f"Errors: {errors}/{SHOTS}")

# Compare with BPOSD
from ldpc import bposd_decoder
from stimbposd.dem_to_matrices import detector_error_model_to_check_matrices

matrices = detector_error_model_to_check_matrices(noisy_dem, allow_undecomposed_hyperedges=True)
bposd = bposd_decoder(
    matrices.check_matrix,
    channel_probs=matrices.priors,
    max_iter=50,
    bp_method="product_sum",
    osd_order=10,
)

bposd_predictions = []
for i in range(SHOTS):
    error_pattern = bposd.decode(det_samples[i].astype(np.uint8))
    obs_dense = matrices.observables_matrix.toarray()
    pred = int(obs_dense @ error_pattern % 2)
    bposd_predictions.append(pred)
bposd_predictions = np.array(bposd_predictions)

bposd_errors = np.sum(bposd_predictions != obs_samples.flatten())
bposd_ler = bposd_errors / SHOTS

print(f"\nBPOSD LER: {bposd_ler:.4f}")
print(f"Errors: {bposd_errors}/{SHOTS}")

print(f"\nRatio: SoftHierarchical / BPOSD = {ler/bposd_ler:.1f}x" if bposd_ler > 0 else "BPOSD perfect")

TEST: SoftHierarchical full decode with proper noise
Samples: 1000 shots
Shots with syndrome: 1000/1000

SoftHierarchical LER: 0.5000
Errors: 500/1000


ValueError: ERROR: OSD order '10' invalid. The 'osd_method' is set to 'OSD_0'. The osd order must therefore be set to 0.

In [14]:
# Debug: Trace the soft information computation step by step
print("="*70)
print("DEBUGGING: Trace soft info computation for one block")
print("="*70)

# Pick a shot with syndrome
test_idx = 0
syndrome = det_samples[test_idx].astype(np.uint8)
true_obs = obs_samples[test_idx]

print(f"Shot {test_idx}: true observable = {true_obs}")

# Find block with syndrome
for block_id in range(soft_hier_noisy.n_blocks):
    det_start, det_stop = soft_hier_noisy.inner_slices[block_id]
    block_syn = syndrome[det_start:det_stop]
    if block_syn.sum() > 0:
        print(f"\nBlock {block_id}: syndrome = {block_syn}, weight = {block_syn.sum()}")
        
        # Get decoder and decode
        decoder = soft_hier_noisy.inner_decoders[block_id]
        obs_matrix = soft_hier_noisy.inner_obs_matrices[block_id]
        
        error_pattern = decoder.decode(block_syn)
        print(f"  Error pattern weight: {error_pattern.sum()}")
        
        # Hard prediction
        obs_dense = obs_matrix.toarray() if hasattr(obs_matrix, 'toarray') else np.asarray(obs_matrix)
        hard_pred = int(obs_dense @ error_pattern % 2)
        print(f"  Hard prediction: {hard_pred}")
        
        # LLRs
        llrs = decoder.log_prob_ratios
        print(f"  LLRs: min={llrs.min():.2f}, max={llrs.max():.2f}")
        
        # Convert LLRs to probabilities
        # LLR = log(P(no error) / P(error)) => P(error) = 1 / (1 + exp(LLR))
        posteriors = 1.0 / (1.0 + np.exp(llrs))
        print(f"  Posteriors: min={posteriors.min():.4f}, max={posteriors.max():.4f}")
        
        # Compute P(logical error) via soft XOR
        logical_support = obs_dense.flatten()
        p_logical = 0.0
        for i in range(min(len(logical_support), len(posteriors))):
            if logical_support[i]:
                p_i = posteriors[i]
                p_logical = p_logical * (1 - p_i) + p_i * (1 - p_logical)
        print(f"  P(logical error) via soft XOR: {p_logical:.4f}")
        
        # What does the decoder's _compute_logical_error_prob_from_llrs return?
        p_from_decoder = soft_hier_noisy._compute_logical_error_prob_from_llrs(
            llrs, obs_matrix, error_pattern
        )
        print(f"  P(logical error) from decoder method: {p_from_decoder:.4f}")
        
        break  # Just one block for now

# Now check what _decode_inner_block returns
print("\n--- Full _decode_inner_block output ---")
hard, p_logical = soft_hier_noisy._decode_inner_block(block_id, block_syn)
print(f"  hard = {hard}, p_logical = {p_logical:.4f}")

DEBUGGING: Trace soft info computation for one block
Shot 0: true observable = [False]

Block 0: syndrome = [0 0 0 1 1 1], weight = 3
  Error pattern weight: 1
  Hard prediction: 0
  LLRs: min=-0.50, max=5.67
  Posteriors: min=0.0034, max=0.6223
  P(logical error) via soft XOR: 0.4469
  P(logical error) from decoder method: 0.4469

--- Full _decode_inner_block output ---
  hard = 0, p_logical = 0.4469


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/1753067461.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hard_pred = int(obs_dense @ error_pattern % 2)


In [15]:
# Debug: Check dimension alignment
print("="*70)
print("DEBUGGING: Check dimension alignment")
print("="*70)

# Get Template DEM matrices for inner code
template_exp = CSSMemoryExperiment(
    code=inner,
    rounds=ROUNDS,
    noise_model=noise,
    basis='Z'
)
template_circuit = template_exp.to_stim()
noisy_template = noise.apply(template_circuit)
template_dem = noisy_template.detector_error_model(decompose_errors=True)

print(f"Template circuit: {noisy_template.num_detectors} detectors, {noisy_template.num_observables} observables")
print(f"Template DEM: {template_dem.num_detectors} detectors, {template_dem.num_observables} observables, {template_dem.num_errors} errors")

# Get matrices
from stimbposd.dem_to_matrices import detector_error_model_to_check_matrices
template_matrices = detector_error_model_to_check_matrices(template_dem, allow_undecomposed_hyperedges=True)

print(f"\nTemplate matrices:")
print(f"  check_matrix shape: {template_matrices.check_matrix.shape}")  # (n_detectors, n_error_vars)
print(f"  observables_matrix shape: {template_matrices.observables_matrix.shape}")  # (n_obs, n_error_vars)
print(f"  priors length: {len(template_matrices.priors)}")

# Compare with actual inner block
block_id = 0
det_start, det_stop = soft_hier_noisy.inner_slices[block_id]
actual_inner_dets = det_stop - det_start

print(f"\nActual inner block:")
print(f"  detectors: {actual_inner_dets}")
print(f"  check_matrix rows: {template_matrices.check_matrix.shape[0]}")

# The issue: Template DEM has more error variables than the block syndrome
# When we decode block_syndrome (6 elements) with a decoder built for 6-detector DEM,
# everything aligns. BUT the observables_matrix has 22 columns (error vars) which
# must match the decoder output.

decoder = soft_hier_noisy.inner_decoders[block_id]
error_pattern = decoder.decode(block_syn)
print(f"\nDecoder output (error_pattern):")
print(f"  shape: {error_pattern.shape}")
print(f"  obs_matrix shape: {soft_hier_noisy.inner_obs_matrices[block_id].shape}")

# The issue is that:
# - Block syndrome has 6 detectors
# - Template DEM has 6 detectors  
# - But Template DEM has 22 error variables (columns in check_matrix)
# - obs_matrix has shape (1, 22)
# - error_pattern has 22 elements (matches!)
# So dimensions align correctly

DEBUGGING: Check dimension alignment
Template circuit: 6 detectors, 1 observables
Template DEM: 6 detectors, 1 observables, 31 errors

Template matrices:
  check_matrix shape: (6, 22)
  observables_matrix shape: (1, 22)
  priors length: 22

Actual inner block:
  detectors: 6
  check_matrix rows: 6

Decoder output (error_pattern):
  shape: (22,)
  obs_matrix shape: (1, 22)


In [16]:
# Debug: Trace the full decode path for one shot
print("="*70)
print("DEBUGGING: Full decode path for one shot")
print("="*70)

test_idx = 0
syndrome = det_samples[test_idx].astype(np.uint8)
true_obs = obs_samples[test_idx]

print(f"Shot {test_idx}: true observable = {true_obs}, syndrome weight = {syndrome.sum()}")

# Step 1: Decode all inner blocks
inner_hard_decisions = np.zeros(soft_hier_noisy.n_blocks, dtype=np.uint8)
inner_logical_probs = np.zeros(soft_hier_noisy.n_blocks, dtype=np.float64)

print("\n--- Step 1: Inner block decoding ---")
for block_id in range(soft_hier_noisy.n_blocks):
    det_start, det_stop = soft_hier_noisy.inner_slices[block_id]
    block_syn = syndrome[det_start:det_stop]
    
    hard, p_logical = soft_hier_noisy._decode_inner_block(block_id, block_syn)
    inner_hard_decisions[block_id] = hard
    inner_logical_probs[block_id] = p_logical
    
    if block_syn.sum() > 0:
        print(f"  Block {block_id}: syn_wt={block_syn.sum()}, hard={hard}, p_log={p_logical:.4f}")

print(f"\nInner hard decisions: {inner_hard_decisions}")
print(f"Inner logical probs: {inner_logical_probs}")

# Step 2: Outer decoding
print("\n--- Step 2: Outer syndrome and decoding ---")
outer_start, outer_stop = soft_hier_noisy.outer_slice
raw_outer_syndrome = syndrome[outer_start:outer_stop].astype(np.uint8)
print(f"Outer slice: {soft_hier_noisy.outer_slice}")
print(f"Raw outer syndrome: {raw_outer_syndrome}, weight = {raw_outer_syndrome.sum()}")

# Inner logical errors flip outer stabilizers
H_outer = soft_hier_noisy.H_outer
print(f"H_outer shape: {H_outer.shape}")

inner_contrib = (inner_hard_decisions @ H_outer.T) % 2
print(f"Inner contribution to outer syndrome: {inner_contrib}")

effective_outer = raw_outer_syndrome ^ inner_contrib.astype(np.uint8)
print(f"Effective outer syndrome: {effective_outer}")

# Check if outer decoder exists
print(f"\nOuter decoder exists: {soft_hier_noisy.outer_decoder is not None}")
if soft_hier_noisy.outer_decoder is not None:
    print(f"Outer decoder type: {type(soft_hier_noisy.outer_decoder).__name__}")
    print(f"Outer base channel probs: {soft_hier_noisy._outer_base_channel_probs}")
    print(f"_has_inner_observables: {soft_hier_noisy._has_inner_observables}")

# Step 3: Final prediction
print("\n--- Step 3: Final prediction ---")
outer_logical_support = soft_hier_noisy.outer_logical_support
print(f"Outer logical support: {outer_logical_support}")

inner_parity = 0
for block_id in outer_logical_support:
    if block_id < soft_hier_noisy.n_blocks:
        inner_parity ^= inner_hard_decisions[block_id]
print(f"Inner parity (from blocks in outer logical support): {inner_parity}")

# Full decode
full_pred = soft_hier_noisy._decode_single_shot(syndrome)
print(f"\nFull prediction: {full_pred}")
print(f"True observable: {int(true_obs[0])}")
print(f"Correct: {full_pred == int(true_obs[0])}")

DEBUGGING: Full decode path for one shot
Shot 0: true observable = [False], syndrome weight = 19

--- Step 1: Inner block decoding ---
  Block 0: syn_wt=3, hard=0, p_log=0.4469
  Block 1: syn_wt=2, hard=0, p_log=0.3612
  Block 2: syn_wt=2, hard=0, p_log=0.1678
  Block 3: syn_wt=2, hard=0, p_log=0.3878
  Block 4: syn_wt=2, hard=0, p_log=0.3612
  Block 6: syn_wt=2, hard=0, p_log=0.3612
  Block 7: syn_wt=1, hard=1, p_log=0.5113
  Block 8: syn_wt=2, hard=0, p_log=0.3648

Inner hard decisions: [0 0 0 0 0 0 0 1 0]
Inner logical probs: [0.44692213 0.36123086 0.16779634 0.38778383 0.36123086 0.01
 0.36123086 0.51131064 0.36484584]

--- Step 2: Outer syndrome and decoding ---
Outer slice: (54, 62)
Raw outer syndrome: [1 1 1 0 0 0 0 0], weight = 3
H_outer shape: (4, 9)
Inner contribution to outer syndrome: [0 0 1 0]


ValueError: operands could not be broadcast together with shapes (8,) (4,) 

In [17]:
# Debug: Investigate the outer syndrome mismatch
print("="*70)
print("DEBUGGING: Outer syndrome dimension mismatch")
print("="*70)

print(f"Outer slice: {soft_hier_noisy.outer_slice}")
print(f"Raw outer syndrome shape: {raw_outer_syndrome.shape}")
print(f"H_outer shape: {soft_hier_noisy.H_outer.shape}")
print(f"n_outer_stabs (from decoder): {soft_hier_noisy.n_outer_stabs}")

# Check outer code
outer_code = soft_hier_noisy.outer_code
print(f"\nOuter code: {outer_code.name}")
print(f"  n={outer_code.n}, k={outer_code.k}")
print(f"  hx shape: {outer_code.hx.shape}")
print(f"  hz shape: {outer_code.hz.shape}")
print(f"  Total stabs: {outer_code.hx.shape[0] + outer_code.hz.shape[0]}")

# The outer syndrome has 8 elements, but H_outer has only 4 rows
# This is because:
# - RotatedSurfaceCode(d=3) has 4 X-stabilizers + 4 Z-stabilizers = 8 total
# - But in Z-basis memory, only Z-stabilizers are measured
# - OR the detector slicing is including both time-like and space-like

# Check experiment slices
print(f"\nSlices from experiment:")
slices = concat_exp.get_detector_slices()
print(f"  outer_dets: {slices['outer_dets']}")
print(f"  total_detectors: {slices['total_detectors']}")
print(f"  basis: {slices['basis']}")

# The issue: For Z-basis memory with rounds=1:
# - We have time-like detectors (comparing rounds) and space-like (comparing final measurement)
# - For rounds=1, there's no previous round, so time-like = 0
# - Space-like detectors = Z-stabilizers for Z-basis
# BUT the outer syndrome has 8 elements, which is 4 (Z-stabs) * 2 (time-like + space-like)?

# Actually for rounds=1:
# - Inner blocks: 6 detectors each (3 Z-stabs × 2 for time-like + space-like?)
# - Outer: 8 detectors

# Let's check how H_outer is built
print(f"\nH_outer matrix:")
print(soft_hier_noisy.H_outer)

DEBUGGING: Outer syndrome dimension mismatch
Outer slice: (54, 62)
Raw outer syndrome shape: (8,)
H_outer shape: (4, 9)
n_outer_stabs (from decoder): 4

Outer code: RotatedSurfaceCode
  n=9, k=1
  hx shape: (4, 9)
  hz shape: (4, 9)
  Total stabs: 8

Slices from experiment:
  outer_dets: 8
  total_detectors: 62
  basis: Z

H_outer matrix:
[[0 0 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]]


In [18]:
# Debug: Understand detector structure in concatenated circuit
print("="*70)
print("DEBUGGING: Detector structure in concatenated circuit")
print("="*70)

# For a CSS code in Z-basis memory:
# - X stabilizers detect Z errors
# - Z stabilizers detect X errors
# 
# For concatenation:
# - Inner blocks: each has (inner_x_stabs + inner_z_stabs) detectors
# - Steane code: 3 X-stabs + 3 Z-stabs = 6 detectors per block
#
# For Z-basis memory (protecting Z logical):
# - Physical X errors → flip Z stabilizers
# - Physical Z errors → flip X stabilizers
# - Inner logical Z error = product of Z on L_z support
# - Inner logical Z error flips which OUTER stabilizers?

# In concatenation, the inner code's n physical qubits become "logical" qubits
# of the outer code. So:
# - Outer X-stabilizer acts on a subset of inner blocks
# - If inner block i has a logical Z error, this is like a "Z error" on 
#   outer qubit i, which flips outer X-stabilizers that include i

# Therefore for Z-basis:
# - Inner Z-type logical errors flip OUTER X-stabilizers
# - H_outer should be outer_code.hx, not outer_code.hz!

print(f"Outer code X-stabilizers (hx): shape {outer_code.hx.shape}")
print(outer_code.hx)
print(f"\nOuter code Z-stabilizers (hz): shape {outer_code.hz.shape}")
print(outer_code.hz)

print(f"\nCurrent H_outer in decoder (using hz):")
print(soft_hier_noisy.H_outer)

# The bug: H_outer uses hz but should use hx for Z-basis
# Because inner Z-logical errors (which affect the Z measurement outcome)
# manifest as outer X-stabilizer violations

# Fix: For Z-basis, use hx; for X-basis, use hz
# (This is the CSS duality)

DEBUGGING: Detector structure in concatenated circuit
Outer code X-stabilizers (hx): shape (4, 9)
[[1 1 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 1 1 0]
 [0 1 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 1]]

Outer code Z-stabilizers (hz): shape (4, 9)
[[0 0 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]]

Current H_outer in decoder (using hz):
[[0 0 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]]


In [19]:
# THEORY: For Z-basis, H_outer should be hx, not hz
# Let's test by manually fixing the decoder and re-running

print("="*70)
print("TEST: Fix H_outer to use hx for Z-basis")
print("="*70)

# Manually patch the decoder to use hx instead of hz
# For Z-basis: Z errors flip X-stabilizers
soft_hier_noisy.H_outer = np.array(outer_code.hx, dtype=np.uint8)
soft_hier_noisy.n_outer_stabs = soft_hier_noisy.H_outer.shape[0]

print(f"Fixed H_outer shape: {soft_hier_noisy.H_outer.shape}")
print(f"Fixed n_outer_stabs: {soft_hier_noisy.n_outer_stabs}")

# Re-test LER
SHOTS = 500
det_samples2, obs_samples2 = sampler.sample(shots=SHOTS, separate_observables=True)

predictions = soft_hier_noisy.decode_batch(det_samples2.astype(np.uint8))
errors = np.sum(predictions.flatten() != obs_samples2.flatten())
ler = errors / SHOTS

print(f"\nSoftHierarchical LER (with hx fix): {ler:.4f}")
print(f"Errors: {errors}/{SHOTS}")

# Check if this is better than random (0.50)
if ler < 0.45:
    print("✓ Better than random!")
else:
    print("✗ Still near random - not the fix")

TEST: Fix H_outer to use hx for Z-basis
Fixed H_outer shape: (4, 9)
Fixed n_outer_stabs: 4

SoftHierarchical LER (with hx fix): 0.5240
Errors: 262/500
✗ Still near random - not the fix


In [20]:
# Debug: Check if outer decoder even exists
print("="*70)
print("DEBUGGING: Outer decoder state")
print("="*70)

print(f"outer_decoder: {soft_hier_noisy.outer_decoder}")
print(f"outer_decoder type: {type(soft_hier_noisy.outer_decoder)}")
print(f"outer_matrices: {soft_hier_noisy.outer_matrices}")
print(f"outer_obs_matrix: {soft_hier_noisy.outer_obs_matrix}")
print(f"_outer_base_channel_probs: {soft_hier_noisy._outer_base_channel_probs}")
print(f"_has_inner_observables: {soft_hier_noisy._has_inner_observables}")

# The issue might be that the outer decoder isn't being built at all!
# Or it's not being used properly

# Check the _build_outer_decoder method behavior
print("\n--- Let's check what _build_outer_decoder does ---")

DEBUGGING: Outer decoder state
outer_decoder: <ldpc._legacy_ldpc_v1._legacy_bposd_decoder.bposd_decoder object at 0x131200520>
outer_decoder type: <class 'ldpc._legacy_ldpc_v1._legacy_bposd_decoder.bposd_decoder'>
outer_matrices: DemMatrices(check_matrix=<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 22 stored elements and shape (8, 13)>, observables_matrix=<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 4 stored elements and shape (1, 13)>, edge_check_matrix=<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 22 stored elements and shape (8, 13)>, edge_observables_matrix=<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 4 stored elements and shape (1, 13)>, hyperedge_to_edge_matrix=<Compressed Sparse Column sparse matrix of dtype 'uint8'
	with 13 stored elements and shape (13, 13)>, priors=array([0.06965543, 0.01      , 0.03613333, 0.01      , 0.03613333,
       0.01      , 0.06965543, 0.02666667, 0.11822827, 0.02666667,
       0.

In [21]:
# Test the renormalization fix
import importlib
from qectostim.decoders import soft_hierarchical_decoder
importlib.reload(soft_hierarchical_decoder)
from qectostim.decoders.soft_hierarchical_decoder import SoftHierarchicalDecoder

print("="*70)
print("TEST: Renormalization fix for Template DEM")
print("="*70)

# Rebuild decoder with reloaded module
soft_hier_fixed = SoftHierarchicalDecoder(
    code=concat,
    dem=noisy_dem,
    rounds=ROUNDS,
    basis='Z'
)

print(f"_inner_decoder_method: {soft_hier_fixed._inner_decoder_method}")
print(f"_has_inner_observables: {soft_hier_fixed._has_inner_observables}")

# Test LER
SHOTS = 500
det_samples_test, obs_samples_test = sampler.sample(shots=SHOTS, separate_observables=True)

predictions_test = soft_hier_fixed.decode_batch(det_samples_test.astype(np.uint8))
errors_test = np.sum(predictions_test.flatten() != obs_samples_test.flatten())
ler_test = errors_test / SHOTS

print(f"\nSoftHierarchical LER (with renormalization fix): {ler_test:.4f}")
print(f"Errors: {errors_test}/{SHOTS}")

if ler_test < 0.45:
    print("✓ Better than random!")
else:
    print("✗ Still near random")

TEST: Renormalization fix for Template DEM
_inner_decoder_method: template_dem
_has_inner_observables: False


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/2372481172.py:12: DeprecationWarning: SoftHierarchicalDecoder is deprecated. Use SoftMessagePassingDecoder instead for more robust hierarchical decoding with adaptive detector slicing.
  soft_hier_fixed = SoftHierarchicalDecoder(



SoftHierarchical LER (with renormalization fix): 0.4620
Errors: 231/500
✗ Still near random


In [22]:
# Debug: Trace renormalization in decode path
print("="*70)
print("DEBUGGING: Trace renormalization")
print("="*70)

# Check if renormalization is happening
print(f"has_accurate_soft_info check:")
print(f"  _has_inner_observables = {soft_hier_fixed._has_inner_observables}")
print(f"  _inner_decoder_method = {soft_hier_fixed._inner_decoder_method}")
print(f"  => has_accurate_soft_info = {soft_hier_fixed._has_inner_observables or soft_hier_fixed._inner_decoder_method == 'template_dem'}")

print(f"\nouter_decoder = {soft_hier_fixed.outer_decoder}")
print(f"_outer_base_channel_probs = {soft_hier_fixed._outer_base_channel_probs}")

# Manually trace a decode
test_idx = 0
syndrome = det_samples_test[test_idx].astype(np.uint8)

# Get inner soft info
inner_probs = []
for block_id in range(soft_hier_fixed.n_blocks):
    det_start, det_stop = soft_hier_fixed.inner_slices[block_id]
    block_syn = syndrome[det_start:det_stop]
    _, p_logical = soft_hier_fixed._decode_inner_block(block_id, block_syn)
    inner_probs.append(p_logical)

print(f"\nInner logical probs: {inner_probs}")
print(f"Mean inner prob: {np.mean(inner_probs):.4f}")

# Check if probs are being used
print(f"\nBase channel probs (first {soft_hier_fixed.n_blocks}): {soft_hier_fixed._outer_base_channel_probs[:soft_hier_fixed.n_blocks]}")
print(f"Inner probs would update these to: {inner_probs}")

DEBUGGING: Trace renormalization
has_accurate_soft_info check:
  _has_inner_observables = False
  _inner_decoder_method = template_dem
  => has_accurate_soft_info = True

outer_decoder = <ldpc._legacy_ldpc_v1._legacy_bposd_decoder.bposd_decoder object at 0x131203070>
_outer_base_channel_probs = [0.06965543 0.01       0.03613333 0.01       0.03613333 0.01
 0.06965543 0.02666667 0.11822827 0.02666667 0.17611187 0.17611187
 0.11822827]

Inner logical probs: [0.01, 0.37328855457566834, 0.378090452486859, 0.44663457286115593, 0.3955197538656247, 0.44692212764341704, 0.36123086294994866, 0.579546310990514, 0.41879781841372743]
Mean inner prob: 0.3789

Base channel probs (first 9): [0.06965543 0.01       0.03613333 0.01       0.03613333 0.01
 0.06965543 0.02666667 0.11822827]
Inner probs would update these to: [0.01, 0.37328855457566834, 0.378090452486859, 0.44663457286115593, 0.3955197538656247, 0.44692212764341704, 0.36123086294994866, 0.579546310990514, 0.41879781841372743]


In [25]:
# Trace FULL decode flow for ONE shot
print("="*70)
print("FULL DECODE TRACE FOR SINGLE SHOT")
print("="*70)

test_idx = 0
syndrome = det_samples_test[test_idx].astype(np.uint8)
actual = obs_samples_test[test_idx, 0]

print(f"Actual logical: {actual}")

# Step 1: Inner decoding
print(f"\n--- STEP 1: Inner Decoding ---")
inner_hard = np.zeros(soft_hier_fixed.n_blocks, dtype=np.uint8)
inner_probs = np.zeros(soft_hier_fixed.n_blocks, dtype=np.float64)

for block_id in range(soft_hier_fixed.n_blocks):
    det_start, det_stop = soft_hier_fixed.inner_slices[block_id]
    block_syn = syndrome[det_start:det_stop]
    hard, p_log = soft_hier_fixed._decode_inner_block(block_id, block_syn)
    inner_hard[block_id] = hard
    inner_probs[block_id] = p_log
    print(f"  Block {block_id}: syn={block_syn}, hard={hard}, p_log={p_log:.4f}")

print(f"\nInner hard: {inner_hard}")
print(f"Inner probs: {inner_probs}")

# Step 2: Renormalization check
print(f"\n--- STEP 2: Renormalization ---")
has_accurate = soft_hier_fixed._has_inner_observables or soft_hier_fixed._inner_decoder_method == 'template_dem'
print(f"has_accurate_soft_info: {has_accurate}")
print(f"outer_decoder: {soft_hier_fixed.outer_decoder is not None}")
print(f"_outer_base_channel_probs: {soft_hier_fixed._outer_base_channel_probs is not None}")

if has_accurate and soft_hier_fixed.outer_decoder and soft_hier_fixed._outer_base_channel_probs is not None:
    print("=> Renormalization WILL happen!")
    updated_probs = soft_hier_fixed._outer_base_channel_probs.copy()
    print(f"Base probs: {updated_probs}")
    
    for block_id in range(min(soft_hier_fixed.n_blocks, len(updated_probs))):
        updated_probs[block_id] = inner_probs[block_id]
    print(f"Updated probs: {updated_probs}")
else:
    print("=> NO renormalization!")

# Step 3: Effective outer syndrome
print(f"\n--- STEP 3: Outer Syndrome ---")
outer_start, outer_stop = soft_hier_fixed.outer_slice
raw_outer = syndrome[outer_start:outer_stop].astype(np.uint8)
print(f"Outer slice: [{outer_start}, {outer_stop})")
print(f"Raw outer syndrome: {raw_outer}")

print(f"H_outer shape: {soft_hier_fixed.H_outer.shape}")
n_blocks_use = min(soft_hier_fixed.n_blocks, soft_hier_fixed.H_outer.shape[1])
n_stabs_use = min(soft_hier_fixed.n_outer_stabs, len(raw_outer))
print(f"n_blocks_use={n_blocks_use}, n_stabs_use={n_stabs_use}")

inner_contrib = (inner_hard[:n_blocks_use] @ soft_hier_fixed.H_outer[:n_stabs_use, :n_blocks_use].T) % 2
print(f"Inner contribution: {inner_contrib}")

effective_syn = raw_outer.copy()
effective_syn[:n_stabs_use] ^= inner_contrib.astype(np.uint8)
print(f"Effective outer syndrome: {effective_syn}")

# Step 4: Outer decode
print(f"\n--- STEP 4: Outer Decode ---")
outer_error = soft_hier_fixed.outer_decoder.decode(effective_syn.astype(np.uint8))
print(f"Outer error pattern: {outer_error}")
print(f"outer_obs_matrix: {soft_hier_fixed.outer_obs_matrix}")
if soft_hier_fixed.outer_obs_matrix is not None:
    print(f"outer_obs_matrix shape: {soft_hier_fixed.outer_obs_matrix.shape}")
    obs_dense = soft_hier_fixed.outer_obs_matrix.toarray() if hasattr(soft_hier_fixed.outer_obs_matrix, 'toarray') else soft_hier_fixed.outer_obs_matrix
    outer_logical = int(obs_dense @ outer_error % 2)
    print(f"Outer logical: {outer_logical}")
else:
    outer_logical = 0
    print(f"No outer_obs_matrix, outer_logical=0")

# Step 5: Final combination
print(f"\n--- STEP 5: Final Combination ---")
inner_parity = 0
print(f"outer_logical_support: {soft_hier_fixed.outer_logical_support}")
for block_id in soft_hier_fixed.outer_logical_support:
    if block_id < soft_hier_fixed.n_blocks:
        inner_parity ^= inner_hard[block_id]
        print(f"  Block {block_id}: inner_hard={inner_hard[block_id]}, cumulative parity={inner_parity}")

final = outer_logical ^ inner_parity
print(f"\nFinal = outer_logical({outer_logical}) XOR inner_parity({inner_parity}) = {final}")
print(f"Actual: {actual}, Match: {final == actual}")

FULL DECODE TRACE FOR SINGLE SHOT
Actual logical: True

--- STEP 1: Inner Decoding ---
  Block 0: syn=[0 0 0 0 0 0], hard=0, p_log=0.0100
  Block 1: syn=[1 0 0 1 1 1], hard=1, p_log=0.3733
  Block 2: syn=[0 0 1 0 0 1], hard=0, p_log=0.3781
  Block 3: syn=[0 1 0 0 0 0], hard=1, p_log=0.4466
  Block 4: syn=[1 0 1 1 0 1], hard=0, p_log=0.3955
  Block 5: syn=[0 0 0 1 1 1], hard=0, p_log=0.4469
  Block 6: syn=[0 1 0 0 1 0], hard=0, p_log=0.3612
  Block 7: syn=[1 1 0 0 0 0], hard=1, p_log=0.5795
  Block 8: syn=[1 0 0 0 1 0], hard=1, p_log=0.4188

Inner hard: [0 1 0 1 0 0 0 1 1]
Inner probs: [0.01       0.37328855 0.37809045 0.44663457 0.39551975 0.44692213
 0.36123086 0.57954631 0.41879782]

--- STEP 2: Renormalization ---
has_accurate_soft_info: True
outer_decoder: True
_outer_base_channel_probs: True
=> Renormalization WILL happen!
Base probs: [0.06965543 0.01       0.03613333 0.01       0.03613333 0.01
 0.06965543 0.02666667 0.11822827 0.02666667 0.17611187 0.17611187
 0.11822827]
Updated

/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/475073588.py:73: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_logical = int(obs_dense @ outer_error % 2)


In [27]:
# Understand the outer decoder structure
print("="*70)
print("OUTER DECODER STRUCTURE ANALYSIS")
print("="*70)

# What attributes exist?
print("Decoder attributes related to outer:")
for attr in dir(soft_hier_fixed):
    if 'outer' in attr.lower():
        val = getattr(soft_hier_fixed, attr, None)
        if not callable(val):
            print(f"  {attr}: {type(val)}")

# What is H_outer built from?
print(f"\nH_outer shape: {soft_hier_fixed.H_outer.shape}")
print(f"H_outer:\n{soft_hier_fixed.H_outer}")

# Compare to outer code
print(f"\nOuter code hx shape: {outer.hx.shape}")
print(f"Outer code hz shape: {outer.hz.shape}")
print(f"Outer code hx (X stabilizers):\n{outer.hx}")
print(f"Outer code hz (Z stabilizers):\n{outer.hz}")

# Observable matrix
print(f"\nouter_obs_matrix shape: {soft_hier_fixed.outer_obs_matrix.shape}")
print(f"outer_obs_matrix:\n{soft_hier_fixed.outer_obs_matrix.toarray()}")

# n_outer_stabs
print(f"\nn_outer_stabs: {soft_hier_fixed.n_outer_stabs}")
print(f"outer_slice: {soft_hier_fixed.outer_slice}")

# Key insight: outer syndrome has 8 elements but H_outer only has 4 rows!
outer_start, outer_stop = soft_hier_fixed.outer_slice
print(f"\nOuter syndrome length: {outer_stop - outer_start}")
print(f"H_outer rows: {soft_hier_fixed.H_outer.shape[0]}")
print(f"MISMATCH: {(outer_stop - outer_start) != soft_hier_fixed.H_outer.shape[0]}")

OUTER DECODER STRUCTURE ANALYSIS
Decoder attributes related to outer:
  H_outer: <class 'numpy.ndarray'>
  _outer_base_channel_probs: <class 'numpy.ndarray'>
  _outer_n_data_errors: <class 'int'>
  n_outer_stabs: <class 'int'>
  outer_code: <class 'qectostim.codes.surface.rotated_surface.RotatedSurfaceCode'>
  outer_decoder: <class 'ldpc._legacy_ldpc_v1._legacy_bposd_decoder.bposd_decoder'>
  outer_logical_support: <class 'set'>
  outer_matrices: <class 'stimbposd.dem_to_matrices.DemMatrices'>
  outer_obs_matrix: <class 'scipy.sparse._csc.csc_matrix'>
  outer_slice: <class 'tuple'>

H_outer shape: (4, 9)
H_outer:
[[0 0 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]]

Outer code hx shape: (4, 9)
Outer code hz shape: (4, 9)
Outer code hx (X stabilizers):
[[1 1 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 1 1 0]
 [0 1 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 1]]
Outer code hz (Z stabilizers):
[[0 0 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 1 0 0 1 0 0 0]]

oute

In [28]:
# KEY INSIGHT: CSS outer code needs BOTH hx AND hz
# - Inner logical X errors flip outer Z stabilizers (hz)
# - Inner logical Z errors flip outer X stabilizers (hx)

# For single logical observable (typically Z), we care about X errors on inner blocks
# Inner X logical errors → data qubit X errors → detected by hz

# But we need FULL H_outer matrix:
# H_outer_full = [hx]  <- X stabilizers detect Z errors
#                [hz]  <- Z stabilizers detect X errors

# The current H_outer = hz only (shape 4x9)
# But outer_syndrome has 8 detectors (4 X + 4 Z)

# Let's see the detector ordering
print("="*70)
print("DETECTOR ORDERING IN OUTER SYNDROME")
print("="*70)

print(f"Outer code stabilizers:")
print(f"  4 X-stabilizers (hx): detect Z errors on data")
print(f"  4 Z-stabilizers (hz): detect X errors on data")

print(f"\nFor the Z-logical observable we track:")
print(f"  Z_L on inner block → Z on outer data qubit")
print(f"  Z errors detected by X-stabilizers (hx)")

print(f"\nBut wait - the inner code logical is also Z-type")
print(f"  Inner Z_L affects outer Z stabilizers? No!")
print(f"  Inner Z_L = Z error on logical qubit = data qubit of outer code")
print(f"  Z error on outer data is detected by outer X stabilizers (hx)")

print(f"\n*** BUG: H_outer should be hx, not hz! ***")

# What about inner X_L?
print(f"\n--- Inner X logical errors ---")
print(f"  Inner X_L = X error on logical = X on outer data")
print(f"  X errors on outer data detected by Z stabilizers (hz)")

# But our decoder only tracks Z logical from inner
# Template DEM observable = inner Z_L
# So inner p_logical is P(Z_L error on inner block)

print(f"\n--- Template DEM gives P(inner Z_L) ---")
print(f"Inner Z_L error → Z error on outer data → detected by outer hx")
print(f"So H_outer should be hx to compute Z's contribution to X stabilizers")
print(f"But we're using hz!!")

DETECTOR ORDERING IN OUTER SYNDROME
Outer code stabilizers:
  4 X-stabilizers (hx): detect Z errors on data
  4 Z-stabilizers (hz): detect X errors on data

For the Z-logical observable we track:
  Z_L on inner block → Z on outer data qubit
  Z errors detected by X-stabilizers (hx)

But wait - the inner code logical is also Z-type
  Inner Z_L affects outer Z stabilizers? No!
  Inner Z_L = Z error on logical qubit = data qubit of outer code
  Z error on outer data is detected by outer X stabilizers (hx)

*** BUG: H_outer should be hx, not hz! ***

--- Inner X logical errors ---
  Inner X_L = X error on logical = X on outer data
  X errors on outer data detected by Z stabilizers (hz)

--- Template DEM gives P(inner Z_L) ---
Inner Z_L error → Z error on outer data → detected by outer hx
So H_outer should be hx to compute Z's contribution to X stabilizers
But we're using hz!!


In [29]:
# Look at the outer_matrices from stimbposd
print("="*70)
print("OUTER DEM MATRICES FROM stimbposd")
print("="*70)

om = soft_hier_fixed.outer_matrices
print(f"Type: {type(om)}")
print(f"check_matrix shape: {om.check_matrix.shape}")
print(f"observables_matrix shape: {om.observables_matrix.shape}")
print(f"priors shape: {om.priors.shape}")

print(f"\ncheck_matrix (H):")
print(om.check_matrix.toarray())

print(f"\nobservables_matrix (L):")
print(om.observables_matrix.toarray())

print(f"\npriors (channel probs):")
print(om.priors)

# So the outer decoder has 8 rows in check_matrix!
# This is what BPOSD uses
# The issue is that H_outer used for syndrome correction only has 4 rows

OUTER DEM MATRICES FROM stimbposd
Type: <class 'stimbposd.dem_to_matrices.DemMatrices'>
check_matrix shape: (8, 13)
observables_matrix shape: (1, 13)
priors shape: (13,)

check_matrix (H):
[[0 0 1 0 0 1 0 1 1 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 1 0 1 1 0 0 0 1 0]
 [0 1 0 0 1 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1]]

observables_matrix (L):
[[1 1 0 1 0 1 0 0 0 0 0 0 0]]

priors (channel probs):
[0.06965543 0.01       0.03613333 0.01       0.03613333 0.01
 0.06965543 0.02666667 0.11822827 0.02666667 0.17611187 0.17611187
 0.11822827]


In [30]:
# DIAGNOSIS: The inner_syndrome_contrib computation is WRONG
# 
# Current code uses H_outer = outer_code.hz (4 rows)
# But outer DEM has 8 detectors!
#
# The outer check_matrix (8 rows) from DEM has:
# - First 4 rows: complex stabilizer combinations
# - Last 4 rows: nearly identity on columns 9-12 (extra errors)
#
# The inner_syndrome_contrib should compute:
#   "What outer syndrome bits flip if inner block i has a logical error?"
#
# This is given by the FIRST n_blocks columns of outer check_matrix!

print("="*70)
print("FIX: Use outer check_matrix instead of H_outer")
print("="*70)

H_outer_correct = soft_hier_fixed.outer_matrices.check_matrix.toarray()[:, :soft_hier_fixed.n_blocks]
print(f"Correct H_outer shape: {H_outer_correct.shape}")
print(f"Correct H_outer:")
print(H_outer_correct)

# Recompute inner_syndrome_contrib with correct H
raw_outer = det_samples_test[test_idx][54:62].astype(np.uint8)
inner_contrib_correct = (inner_hard @ H_outer_correct.T) % 2
print(f"\nInner hard: {inner_hard}")
print(f"Inner contrib (CORRECT): {inner_contrib_correct}")
print(f"Inner contrib (old buggy): {(inner_hard[:9] @ soft_hier_fixed.H_outer[:4, :9].T) % 2}")

effective_correct = raw_outer ^ inner_contrib_correct.astype(np.uint8)
print(f"\nRaw outer syndrome: {raw_outer}")
print(f"Effective syndrome (CORRECT): {effective_correct}")
print(f"Effective syndrome (buggy): {effective_syn}")

# Now decode with correct effective syndrome
outer_error_correct = soft_hier_fixed.outer_decoder.decode(effective_correct.astype(np.uint8))
print(f"\nOuter error (correct): {outer_error_correct}")
obs_dense = soft_hier_fixed.outer_obs_matrix.toarray()
outer_log_correct = int(obs_dense @ outer_error_correct % 2)
print(f"Outer logical (correct): {outer_log_correct}")

# Final with correct
final_correct = outer_log_correct ^ inner_parity
print(f"\nFinal prediction (correct): {final_correct}")
print(f"Actual: {actual}")
print(f"Match: {final_correct == actual}")

FIX: Use outer check_matrix instead of H_outer
Correct H_outer shape: (8, 9)
Correct H_outer:
[[0 0 1 0 0 1 0 1 1]
 [1 0 1 1 0 0 0 0 0]
 [0 0 0 1 1 0 1 1 0]
 [0 1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]

Inner hard: [0 1 0 1 0 0 0 1 1]
Inner contrib (CORRECT): [0 1 0 1 1 0 0 0]
Inner contrib (old buggy): [1 0 0 0]

Raw outer syndrome: [0 0 1 1 0 1 0 1]
Effective syndrome (CORRECT): [0 1 1 0 1 1 0 1]
Effective syndrome (buggy): [1 0 1 1 0 1 0 1]

Outer error (correct): [0 0 1 1 1 0 1 0 1 0 1 0 1]
Outer logical (correct): 1

Final prediction (correct): 1
Actual: True
Match: True


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/1557605424.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_log_correct = int(obs_dense @ outer_error_correct % 2)


In [31]:
# Test the fix on all shots
print("="*70)
print("TEST FIX ON ALL SHOTS")
print("="*70)

# Get correct H_outer
H_outer_correct = soft_hier_fixed.outer_matrices.check_matrix.toarray()[:, :soft_hier_fixed.n_blocks].astype(np.uint8)

def decode_with_fix(decoder, syndrome, H_outer_correct):
    """Decode with corrected H_outer."""
    # Step 1: Inner decoding
    inner_hard = np.zeros(decoder.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(decoder.n_blocks, dtype=np.float64)
    
    for block_id in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[block_id]
        block_syn = syndrome[det_start:det_stop]
        hard, p_log = decoder._decode_inner_block(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    # Step 2: Renormalization
    updated_probs = decoder._outer_base_channel_probs.copy()
    for block_id in range(min(decoder.n_blocks, len(updated_probs))):
        updated_probs[block_id] = inner_probs[block_id]
    decoder.outer_decoder.update_channel_probs(updated_probs)
    
    # Step 3: Effective outer syndrome with CORRECT H_outer
    outer_start, outer_stop = decoder.outer_slice
    raw_outer = syndrome[outer_start:outer_stop].astype(np.uint8)
    inner_contrib = (inner_hard @ H_outer_correct.T) % 2
    effective = raw_outer ^ inner_contrib.astype(np.uint8)
    
    # Step 4: Decode outer
    outer_error = decoder.outer_decoder.decode(effective.astype(np.uint8))
    obs_dense = decoder.outer_obs_matrix.toarray() if hasattr(decoder.outer_obs_matrix, 'toarray') else decoder.outer_obs_matrix
    outer_logical = int(obs_dense @ outer_error % 2)
    if hasattr(outer_logical, '__len__'):
        outer_logical = int(outer_logical[0])
    
    # Step 5: Combine
    inner_parity = 0
    for block_id in decoder.outer_logical_support:
        if block_id < decoder.n_blocks:
            inner_parity ^= inner_hard[block_id]
    
    return outer_logical ^ inner_parity

# Test
errors_fixed = 0
for i in range(len(det_samples_test)):
    pred = decode_with_fix(soft_hier_fixed, det_samples_test[i].astype(np.uint8), H_outer_correct)
    actual = obs_samples_test[i, 0]
    if pred != actual:
        errors_fixed += 1

ler_fixed = errors_fixed / len(det_samples_test)
print(f"Fixed LER: {ler_fixed:.4f} ({errors_fixed}/{len(det_samples_test)} errors)")

# Compare to original
ler_orig = error_count_fixed / len(det_samples_test)
print(f"Original LER: {ler_orig:.4f}")

if ler_fixed < 0.1:
    print("✓ FIX WORKS! Much better than random.")
else:
    print("✗ Still high LER.")

TEST FIX ON ALL SHOTS


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/3490747254.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_logical = int(obs_dense @ outer_error % 2)


Fixed LER: 0.4980 (249/500 errors)


NameError: name 'error_count_fixed' is not defined

In [33]:
# Debug: Is the outer_logical_support correct?
print("="*70)
print("DEBUG: outer_logical_support")
print("="*70)

print(f"outer_logical_support: {soft_hier_fixed.outer_logical_support}")
print(f"This says ALL blocks are in the logical support!")

# The outer observable from DEM
obs_mat = soft_hier_fixed.outer_obs_matrix.toarray()
print(f"\nOuter observable (from DEM): {obs_mat}")
print(f"Non-zero positions: {np.where(obs_mat[0] != 0)[0]}")

# Positions 0, 1, 3, 5 have value 1
# These correspond to inner blocks 0, 1, 3, 5
# So the logical is: block0 XOR block1 XOR block3 XOR block5

# But outer_logical_support is {0,1,2,3,4,5,6,7,8} = ALL blocks!
print(f"\n*** BUG: outer_logical_support should only be blocks in observable support ***")

# Let's compute what the correct support should be
correct_support = set(np.where(obs_mat[0, :soft_hier_fixed.n_blocks] != 0)[0])
print(f"Correct support: {correct_support}")

DEBUG: outer_logical_support
outer_logical_support: {0, 1, 2, 3, 4, 5, 6, 7, 8}
This says ALL blocks are in the logical support!

Outer observable (from DEM): [[1 1 0 1 0 1 0 0 0 0 0 0 0]]
Non-zero positions: [0 1 3 5]

*** BUG: outer_logical_support should only be blocks in observable support ***
Correct support: {np.int64(0), np.int64(1), np.int64(3), np.int64(5)}


In [34]:
# Test with BOTH fixes:
# 1. Correct H_outer from DEM check_matrix
# 2. Correct outer_logical_support from DEM observable

print("="*70)
print("TEST WITH BOTH FIXES")
print("="*70)

# Correct H_outer
H_outer_fix = soft_hier_fixed.outer_matrices.check_matrix.toarray()[:, :soft_hier_fixed.n_blocks].astype(np.uint8)

# Correct logical support  
obs_mat = soft_hier_fixed.outer_obs_matrix.toarray()
logical_support_fix = set(np.where(obs_mat[0, :soft_hier_fixed.n_blocks] != 0)[0])

print(f"Fixed H_outer shape: {H_outer_fix.shape}")
print(f"Fixed logical support: {logical_support_fix}")

def decode_with_both_fixes(decoder, syndrome, H_outer_fix, logical_support_fix):
    """Decode with both corrections."""
    # Step 1: Inner decoding
    inner_hard = np.zeros(decoder.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(decoder.n_blocks, dtype=np.float64)
    
    for block_id in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[block_id]
        block_syn = syndrome[det_start:det_stop]
        hard, p_log = decoder._decode_inner_block(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    # Step 2: Renormalization
    updated_probs = decoder._outer_base_channel_probs.copy()
    for block_id in range(min(decoder.n_blocks, len(updated_probs))):
        updated_probs[block_id] = inner_probs[block_id]
    decoder.outer_decoder.update_channel_probs(updated_probs)
    
    # Step 3: Effective outer syndrome with CORRECT H_outer
    outer_start, outer_stop = decoder.outer_slice
    raw_outer = syndrome[outer_start:outer_stop].astype(np.uint8)
    inner_contrib = (inner_hard @ H_outer_fix.T) % 2
    effective = raw_outer ^ inner_contrib.astype(np.uint8)
    
    # Step 4: Decode outer
    outer_error = decoder.outer_decoder.decode(effective.astype(np.uint8))
    obs_dense = decoder.outer_obs_matrix.toarray() if hasattr(decoder.outer_obs_matrix, 'toarray') else decoder.outer_obs_matrix
    outer_logical = int(obs_dense @ outer_error % 2)
    if hasattr(outer_logical, '__len__'):
        outer_logical = int(outer_logical[0])
    
    # Step 5: Combine with CORRECT logical support
    inner_parity = 0
    for block_id in logical_support_fix:
        if block_id < decoder.n_blocks:
            inner_parity ^= inner_hard[block_id]
    
    return outer_logical ^ inner_parity

# Test
errors_both_fixes = 0
for i in range(len(det_samples_test)):
    pred = decode_with_both_fixes(soft_hier_fixed, det_samples_test[i].astype(np.uint8), H_outer_fix, logical_support_fix)
    actual = obs_samples_test[i, 0]
    if pred != actual:
        errors_both_fixes += 1

ler_both = errors_both_fixes / len(det_samples_test)
print(f"\nLER with BOTH fixes: {ler_both:.4f} ({errors_both_fixes}/{len(det_samples_test)} errors)")

if ler_both < 0.1:
    print("✓ BOTH FIXES WORK! Much better than random.")
elif ler_both < 0.3:
    print("~ Improved but not great")
else:
    print("✗ Still high LER, more bugs?")

TEST WITH BOTH FIXES
Fixed H_outer shape: (8, 9)
Fixed logical support: {np.int64(0), np.int64(1), np.int64(3), np.int64(5)}


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/3511614239.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_logical = int(obs_dense @ outer_error % 2)



LER with BOTH fixes: 0.5060 (253/500 errors)
✗ Still high LER, more bugs?


In [35]:
# Let's try simpler approaches - maybe the combination is wrong

print("="*70)
print("TEST DIFFERENT COMBINATION STRATEGIES")
print("="*70)

def decode_variants(decoder, syndrome, H_outer_fix, logical_support_fix):
    """Try different decode strategies."""
    # Inner decoding
    inner_hard = np.zeros(decoder.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(decoder.n_blocks, dtype=np.float64)
    
    for block_id in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[block_id]
        block_syn = syndrome[det_start:det_stop]
        hard, p_log = decoder._decode_inner_block(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    # Renormalization
    updated_probs = decoder._outer_base_channel_probs.copy()
    for block_id in range(min(decoder.n_blocks, len(updated_probs))):
        updated_probs[block_id] = inner_probs[block_id]
    decoder.outer_decoder.update_channel_probs(updated_probs)
    
    # Effective outer syndrome
    outer_start, outer_stop = decoder.outer_slice
    raw_outer = syndrome[outer_start:outer_stop].astype(np.uint8)
    inner_contrib = (inner_hard @ H_outer_fix.T) % 2
    effective = raw_outer ^ inner_contrib.astype(np.uint8)
    
    # Decode outer
    outer_error = decoder.outer_decoder.decode(effective.astype(np.uint8))
    obs_dense = decoder.outer_obs_matrix.toarray()
    outer_logical = int(obs_dense @ outer_error % 2)
    if hasattr(outer_logical, '__len__'):
        outer_logical = int(outer_logical[0])
    
    # Inner parity variants
    inner_parity_all = np.sum(inner_hard) % 2  # All blocks
    inner_parity_obs = 0
    for b in logical_support_fix:
        if b < len(inner_hard):
            inner_parity_obs ^= inner_hard[b]
    
    return {
        'outer_only': outer_logical,
        'outer_xor_all': outer_logical ^ inner_parity_all,
        'outer_xor_obs': outer_logical ^ inner_parity_obs,
        'inner_parity_all': inner_parity_all,
        'inner_parity_obs': inner_parity_obs,
    }

# Also try without effective syndrome correction
def decode_no_effective(decoder, syndrome):
    """Try without effective syndrome - just use raw outer."""
    # Inner decoding
    inner_hard = np.zeros(decoder.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(decoder.n_blocks, dtype=np.float64)
    
    for block_id in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[block_id]
        block_syn = syndrome[det_start:det_stop]
        hard, p_log = decoder._decode_inner_block(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    # Renormalization
    updated_probs = decoder._outer_base_channel_probs.copy()
    for block_id in range(min(decoder.n_blocks, len(updated_probs))):
        updated_probs[block_id] = inner_probs[block_id]
    decoder.outer_decoder.update_channel_probs(updated_probs)
    
    # Raw outer syndrome (NO inner_contrib correction)
    outer_start, outer_stop = decoder.outer_slice
    raw_outer = syndrome[outer_start:outer_stop].astype(np.uint8)
    
    # Decode outer
    outer_error = decoder.outer_decoder.decode(raw_outer)
    obs_dense = decoder.outer_obs_matrix.toarray()
    outer_logical = int(obs_dense @ outer_error % 2)
    if hasattr(outer_logical, '__len__'):
        outer_logical = int(outer_logical[0])
    
    return outer_logical

# Test all variants
results = {'outer_only': 0, 'outer_xor_all': 0, 'outer_xor_obs': 0, 'no_effective': 0}

for i in range(len(det_samples_test)):
    syn = det_samples_test[i].astype(np.uint8)
    actual = obs_samples_test[i, 0]
    
    v = decode_variants(soft_hier_fixed, syn, H_outer_fix, logical_support_fix)
    n = decode_no_effective(soft_hier_fixed, syn)
    
    if v['outer_only'] != actual:
        results['outer_only'] += 1
    if v['outer_xor_all'] != actual:
        results['outer_xor_all'] += 1
    if v['outer_xor_obs'] != actual:
        results['outer_xor_obs'] += 1
    if n != actual:
        results['no_effective'] += 1

print(f"LER variants:")
for k, v in results.items():
    ler = v / len(det_samples_test)
    status = "✓" if ler < 0.1 else "~" if ler < 0.3 else "✗"
    print(f"  {k}: {ler:.4f} ({v}/{len(det_samples_test)}) {status}")

TEST DIFFERENT COMBINATION STRATEGIES
LER variants:
  outer_only: 0.4900 (245/500) ✗
  outer_xor_all: 0.4980 (249/500) ✗
  outer_xor_obs: 0.5060 (253/500) ✗
  no_effective: 0.4320 (216/500) ✗


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/1867549132.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_logical = int(obs_dense @ outer_error % 2)
/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/1867549132.py:81: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outer_logical = int(obs_dense @ outer_error % 2)


In [36]:
# Test: Is the outer BPOSD decoder working at all?
# Let's test it with perfect inner information

print("="*70)
print("TEST: Outer BPOSD with perfect inner info")
print("="*70)

# Create fresh outer decoder with base probs
from ldpc import bposd_decoder

outer_H = soft_hier_fixed.outer_matrices.check_matrix.toarray()
outer_L = soft_hier_fixed.outer_matrices.observables_matrix.toarray()
outer_priors = soft_hier_fixed.outer_matrices.priors.copy()

print(f"Outer H shape: {outer_H.shape}")
print(f"Outer L shape: {outer_L.shape}")
print(f"Outer priors: {outer_priors}")

# Create decoder
outer_dec = bposd_decoder(
    outer_H,
    error_rate=0.1,  # placeholder, we'll update
    bp_method="ms",
    max_iter=100,
    osd_method="osd_cs",
    osd_order=10
)

# Test: just decode outer syndromes directly with base priors
outer_dec.update_channel_probs(outer_priors)

errors_outer_direct = 0
for i in range(len(det_samples_test)):
    outer_start, outer_stop = soft_hier_fixed.outer_slice
    outer_syn = det_samples_test[i, outer_start:outer_stop].astype(np.uint8)
    
    outer_error = outer_dec.decode(outer_syn)
    pred = int(outer_L @ outer_error % 2)
    if hasattr(pred, '__len__'):
        pred = int(pred[0])
    
    actual = obs_samples_test[i, 0]
    if pred != actual:
        errors_outer_direct += 1

ler_outer_direct = errors_outer_direct / len(det_samples_test)
print(f"\nOuter BPOSD direct (no inner correction): LER = {ler_outer_direct:.4f}")

# What about just inner decoding?
errors_inner_only = 0
for i in range(len(det_samples_test)):
    syn = det_samples_test[i].astype(np.uint8)
    
    # Decode all inner blocks
    inner_logicals = []
    for block_id in range(soft_hier_fixed.n_blocks):
        det_start, det_stop = soft_hier_fixed.inner_slices[block_id]
        block_syn = syn[det_start:det_stop]
        hard, _ = soft_hier_fixed._decode_inner_block(block_id, block_syn)
        inner_logicals.append(hard)
    
    # Just use inner parity on observable support
    obs_support = [0, 1, 3, 5]  # From DEM observable
    pred = sum(inner_logicals[b] for b in obs_support) % 2
    
    actual = obs_samples_test[i, 0]
    if pred != actual:
        errors_inner_only += 1

ler_inner_only = errors_inner_only / len(det_samples_test)
print(f"Inner-only (parity on obs support): LER = {ler_inner_only:.4f}")

TEST: Outer BPOSD with perfect inner info
Outer H shape: (8, 13)
Outer L shape: (1, 13)
Outer priors: [0.06965543 0.01       0.03613333 0.01       0.03613333 0.01
 0.06965543 0.02666667 0.11822827 0.02666667 0.17611187 0.17611187
 0.11822827]

Outer BPOSD direct (no inner correction): LER = 0.4600
Inner-only (parity on obs support): LER = 0.5140


/var/folders/wb/b8rfjwj12jn7gld5g0b3c2x80000gn/T/ipykernel_5243/2058746596.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(outer_L @ outer_error % 2)


In [43]:
# Check baseline BPOSD - it decodes the FULL concatenated DEM
print("="*70)
print("BASELINE BPOSD ANALYSIS")
print("="*70)

# Regenerate DEM
dem = noisy_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

# What DEM does baseline BPOSD use?
print(f"Full concatenated DEM:")
print(f"  num_detectors: {dem.num_detectors}")
print(f"  num_observables: {dem.num_observables}")

# Get the full matrices using the function
from stimbposd.dem_to_matrices import detector_error_model_to_check_matrices
full_matrices = detector_error_model_to_check_matrices(dem)
print(f"\nFull check_matrix shape: {full_matrices.check_matrix.shape}")
print(f"Full observable_matrix shape: {full_matrices.observables_matrix.shape}")
print(f"Full priors shape: {full_matrices.priors.shape}")

# What does our outer decoder have?
print(f"\nOur outer decoder:")
print(f"  check_matrix shape: {soft_hier_fixed.outer_matrices.check_matrix.shape}")

# The fundamental issue:
print(f"\n*** ARCHITECTURAL PROBLEM ***")
print(f"BPOSD uses ALL {dem.num_detectors} detectors = full {full_matrices.check_matrix.shape}")
print(f"Our outer DEM only has {soft_hier_fixed.outer_matrices.check_matrix.shape[0]} detectors")
print(f"\nThe hierarchical decoder tries to:")
print(f"  1. Decode inner blocks separately (6 dets each × 9 blocks = 54 dets)")
print(f"  2. Decode outer part (8 dets)")
print(f"  3. Combine results")
print(f"\nBut the outer DEM was built from the CONCATENATED circuit!")
print(f"The outer decoder sees a problem it can't solve alone.")

BASELINE BPOSD ANALYSIS
Full concatenated DEM:
  num_detectors: 62
  num_observables: 1

Full check_matrix shape: (62, 274)
Full observable_matrix shape: (1, 274)
Full priors shape: (274,)

Our outer decoder:
  check_matrix shape: (8, 13)

*** ARCHITECTURAL PROBLEM ***
BPOSD uses ALL 62 detectors = full (62, 274)
Our outer DEM only has 8 detectors

The hierarchical decoder tries to:
  1. Decode inner blocks separately (6 dets each × 9 blocks = 54 dets)
  2. Decode outer part (8 dets)
  3. Combine results

But the outer DEM was built from the CONCATENATED circuit!
The outer decoder sees a problem it can't solve alone.


In [44]:
# Let's understand the TRUE relationship between inner logical errors and outer detectors
# by looking at the actual concatenated DEM structure

print("="*70)
print("ANALYZING ACTUAL DEM STRUCTURE")
print("="*70)

# Find errors in full DEM that have inner observables
# The DEM was generated without inner observables (num_observables=1)
# So we need to look at the TEMPLATE DEM which has inner observables

# Actually, let's check what errors affect outer detectors
outer_start, outer_stop = soft_hier_fixed.outer_slice
print(f"Outer detector range: [{outer_start}, {outer_stop})")

# Find all errors touching outer detectors
errors_touching_outer = []
for instr in dem.flattened():
    if instr.type != "error":
        continue
    prob = instr.args_copy()[0]
    targets = instr.targets_copy()
    
    outer_dets = []
    inner_dets = []
    has_main_obs = False
    
    for t in targets:
        if t.is_relative_detector_id():
            if outer_start <= t.val < outer_stop:
                outer_dets.append(t.val - outer_start)
            else:
                inner_dets.append(t.val)
        elif t.is_logical_observable_id() and t.val == 0:
            has_main_obs = True
    
    if outer_dets:
        errors_touching_outer.append({
            'prob': prob,
            'outer_dets': outer_dets,
            'inner_dets': inner_dets,
            'has_obs': has_main_obs
        })

print(f"\nFound {len(errors_touching_outer)} errors touching outer detectors")

# Show some
for i, e in enumerate(errors_touching_outer[:10]):
    print(f"  {i}: p={e['prob']:.4f}, outer={e['outer_dets']}, inner={e['inner_dets'][:5]}..., obs={e['has_obs']}")

# Key insight: errors that touch BOTH inner and outer detectors
mixed_errors = [e for e in errors_touching_outer if e['inner_dets']]
print(f"\nErrors touching BOTH inner and outer: {len(mixed_errors)}")

pure_outer_errors = [e for e in errors_touching_outer if not e['inner_dets']]
print(f"Pure outer errors: {len(pure_outer_errors)}")

ANALYZING ACTUAL DEM STRUCTURE
Outer detector range: [54, 62)

Found 137 errors touching outer detectors
  0: p=0.0267, outer=[1], inner=[0, 1]..., obs=True
  1: p=0.0519, outer=[1], inner=[1]..., obs=True
  2: p=0.0640, outer=[1], inner=[1, 2]..., obs=True
  3: p=0.0267, outer=[5], inner=[1, 2]..., obs=True
  4: p=0.0640, outer=[1], inner=[2]..., obs=True
  5: p=0.0135, outer=[1], inner=[2, 1]..., obs=True
  6: p=0.0267, outer=[5], inner=[2, 4]..., obs=True
  7: p=0.0267, outer=[1], inner=[4]..., obs=True
  8: p=0.0622, outer=[5], inner=[4]..., obs=True
  9: p=0.0622, outer=[5], inner=[4, 5]..., obs=True

Errors touching BOTH inner and outer: 127
Pure outer errors: 10


In [45]:
# APPROACH: Build synthetic outer decoder from outer code structure
# 
# The key insight from the literature (Poulin 2006, etc.):
# - Inner decoder gives P(logical_i) for each block
# - Outer decoder sees "effective channel" where each data qubit has error prob P(logical_i)
# - Outer stabilizers detect combinations of these effective errors
#
# But the OUTER SYNDROME from the concatenated circuit is NOT the syndrome of this effective channel!
# It's the syndrome of the PHYSICAL circuit.
#
# The hierarchical algorithm needs to:
# 1. Decode inner blocks → hard decisions + soft info
# 2. Correct outer syndrome: subtract contribution from inner hard decisions
# 3. Decode corrected outer syndrome with updated channel probs

print("="*70)
print("TEST: Synthetic outer code decoder")
print("="*70)

# Build outer decoder purely from outer code structure
# H_outer = hx (for Z-logical) or hz (for X-logical)
# For Z-logical, we track X errors on data qubits
# X errors are detected by Z stabilizers (hz)

# Actually wait - what basis are we decoding?
print(f"Decoder basis: {soft_hier_fixed.basis}")

# For Z-basis decoding:
# - We track Z errors (which flip X stabilizers)
# - Inner Z-logical errors are Z errors on outer data qubits  
# - Z errors on outer data detected by X stabilizers (hx)

# Let's build a proper outer decoder
from ldpc import bposd_decoder

# Full parity check for CSS: stack hx and hz
H_full = np.vstack([outer.hx, outer.hz])  # (8, 9)
print(f"H_full shape: {H_full.shape}")

# For Z-logical (tracked observable):
# Logical Z of outer code
logical_z = np.zeros((1, 9), dtype=np.uint8)
for i in range(9):
    # Logical Z has support... let me check
    pass

# Actually, the outer observable from DEM should tell us
print(f"\nOuter observable (from DEM):")
print(soft_hier_fixed.outer_obs_matrix.toarray())
# [1 1 0 1 0 1 0 0 0 0 0 0 0] 
# Positions 0,1,3,5 = first 4 blocks... but wait, this is error positions, not qubit positions

# The observable is on ERROR VARIABLES not DATA QUBITS
# Error vars 0-8 are inner logical errors
# So observable = block 0 XOR block 1 XOR block 3 XOR block 5

# This makes sense for a d=3 surface code where the logical is a string of 4 qubits
print(f"\nFor d=3 rotated surface code:")
print(f"  9 data qubits in 3x3 grid")
print(f"  Logical Z = column of qubits")
print(f"  Support = 3 qubits... but observable has 4 blocks?")

# Let me check the outer code logical
print(f"\nOuter code logicals:")
print(f"  outer.k = {outer.k}")
# We need to find logical Z support

TEST: Synthetic outer code decoder
Decoder basis: Z
H_full shape: (8, 9)

Outer observable (from DEM):
[[1 1 0 1 0 1 0 0 0 0 0 0 0]]

For d=3 rotated surface code:
  9 data qubits in 3x3 grid
  Logical Z = column of qubits
  Support = 3 qubits... but observable has 4 blocks?

Outer code logicals:
  outer.k = 1


In [46]:
# SIMPLEST APPROACH: Just use inner soft info for weighted prediction
# This ignores the outer decoder entirely but should work if Template DEM is good

print("="*70)
print("TEST: Weighted Inner-Only Decoding")
print("="*70)

# The observable has support on blocks 0, 1, 3, 5
obs_support = [0, 1, 3, 5]

def decode_weighted_inner_only(decoder, syndrome, obs_support):
    """Decode using only inner soft info, weighted by confidence."""
    # Decode all inner blocks
    inner_hard = np.zeros(decoder.n_blocks, dtype=np.uint8)
    inner_probs = np.zeros(decoder.n_blocks, dtype=np.float64)  # P(logical error)
    
    for block_id in range(decoder.n_blocks):
        det_start, det_stop = decoder.inner_slices[block_id]
        block_syn = syndrome[det_start:det_stop]
        hard, p_log = decoder._decode_inner_block(block_id, block_syn)
        inner_hard[block_id] = hard
        inner_probs[block_id] = p_log
    
    # Weighted vote based on soft info
    # For blocks in observable support, compute weighted prediction
    total_weight = 0
    weighted_sum = 0
    
    for b in obs_support:
        if b < len(inner_hard):
            # Weight = confidence = 1 - 2*min(p, 1-p)
            # More confident when p close to 0 or 1
            p = inner_probs[b]
            confidence = abs(1 - 2 * p)  # 0 at p=0.5, 1 at p=0 or p=1
            
            # Vote = hard decision weighted by confidence
            weighted_sum += inner_hard[b] * confidence
            total_weight += confidence
    
    # Threshold at 0.5
    if total_weight > 0:
        weighted_avg = weighted_sum / total_weight
        return int(weighted_avg > 0.5)
    else:
        # Fallback to simple parity
        return sum(inner_hard[b] for b in obs_support if b < len(inner_hard)) % 2

# Test
errors_weighted = 0
for i in range(len(det_samples_test)):
    pred = decode_weighted_inner_only(soft_hier_fixed, det_samples_test[i].astype(np.uint8), obs_support)
    actual = obs_samples_test[i, 0]
    if pred != actual:
        errors_weighted += 1

ler_weighted = errors_weighted / len(det_samples_test)
print(f"Weighted inner-only LER: {ler_weighted:.4f}")

# Also try simple XOR (no weighting)
errors_simple_xor = 0
for i in range(len(det_samples_test)):
    syn = det_samples_test[i].astype(np.uint8)
    inner_hard = []
    for block_id in range(soft_hier_fixed.n_blocks):
        det_start, det_stop = soft_hier_fixed.inner_slices[block_id]
        block_syn = syn[det_start:det_stop]
        hard, _ = soft_hier_fixed._decode_inner_block(block_id, block_syn)
        inner_hard.append(hard)
    
    pred = sum(inner_hard[b] for b in obs_support if b < len(inner_hard)) % 2
    if pred != obs_samples_test[i, 0]:
        errors_simple_xor += 1

ler_simple_xor = errors_simple_xor / len(det_samples_test)
print(f"Simple XOR on obs support: {ler_simple_xor:.4f}")

TEST: Weighted Inner-Only Decoding
Weighted inner-only LER: 0.4720
Simple XOR on obs support: 0.5140


In [47]:
# Check inner decoding quality
print("="*70)
print("INNER DECODING QUALITY CHECK")
print("="*70)

# For this we need to know the TRUE inner logical errors
# But the concatenated DEM only has 1 observable!
# We don't have ground truth for individual inner blocks

# Wait - we CAN generate ground truth by looking at the error propagation
# An inner logical error on block i means the inner code's Z-logical was flipped

# Actually let's check if the inner decoders are even remotely correct
# by looking at syndrome patterns

# Check a few shots
for test_idx in range(5):
    syn = det_samples_test[test_idx].astype(np.uint8)
    actual_obs = obs_samples_test[test_idx, 0]
    
    print(f"\n--- Shot {test_idx} (actual={actual_obs}) ---")
    
    inner_results = []
    for block_id in range(soft_hier_fixed.n_blocks):
        det_start, det_stop = soft_hier_fixed.inner_slices[block_id]
        block_syn = syn[det_start:det_stop]
        hard, p_log = soft_hier_fixed._decode_inner_block(block_id, block_syn)
        inner_results.append((hard, p_log, np.sum(block_syn)))
        
    # Show
    print(f"Block results (hard, p_log, syn_weight):")
    for b, (h, p, w) in enumerate(inner_results):
        marker = "*" if b in obs_support else " "
        print(f"  {marker}Block {b}: hard={h}, p_log={p:.3f}, syn_wt={w}")
    
    pred_xor = sum(inner_results[b][0] for b in obs_support) % 2
    print(f"XOR on obs_support: {pred_xor}, actual: {actual_obs}, match: {pred_xor == actual_obs}")

INNER DECODING QUALITY CHECK

--- Shot 0 (actual=True) ---
Block results (hard, p_log, syn_weight):
  *Block 0: hard=0, p_log=0.010, syn_wt=0
  *Block 1: hard=1, p_log=0.373, syn_wt=4
   Block 2: hard=0, p_log=0.378, syn_wt=2
  *Block 3: hard=1, p_log=0.447, syn_wt=1
   Block 4: hard=0, p_log=0.396, syn_wt=4
  *Block 5: hard=0, p_log=0.447, syn_wt=3
   Block 6: hard=0, p_log=0.361, syn_wt=2
   Block 7: hard=1, p_log=0.580, syn_wt=2
   Block 8: hard=1, p_log=0.419, syn_wt=2
XOR on obs_support: 0, actual: True, match: False

--- Shot 1 (actual=True) ---
Block results (hard, p_log, syn_weight):
  *Block 0: hard=0, p_log=0.010, syn_wt=0
  *Block 1: hard=1, p_log=0.485, syn_wt=2
   Block 2: hard=1, p_log=0.447, syn_wt=4
  *Block 3: hard=1, p_log=0.461, syn_wt=3
   Block 4: hard=0, p_log=0.361, syn_wt=2
  *Block 5: hard=0, p_log=0.361, syn_wt=2
   Block 6: hard=0, p_log=0.010, syn_wt=0
   Block 7: hard=1, p_log=0.526, syn_wt=2
   Block 8: hard=0, p_log=0.378, syn_wt=2
XOR on obs_support: 0, 

In [48]:
# Check the actual observable from the FULL concatenated DEM
print("="*70)
print("OBSERVABLE ANALYSIS")
print("="*70)

print("Full DEM observable matrix shape:", full_matrices.observables_matrix.shape)
print("Full DEM observable matrix:")
obs_full = full_matrices.observables_matrix.toarray()[0]
print(f"Non-zero positions (of {len(obs_full)}): {np.where(obs_full != 0)[0]}")

# What do these positions correspond to?
# The full DEM has 274 error variables
# Need to map back to understand what errors contribute to the observable

print(f"\nOur outer DEM observable:")
print(soft_hier_fixed.outer_obs_matrix.toarray()[0])

# The outer observable was built from errors that flip L0 in the OUTER slice
# But the full observable is much more complex!

OBSERVABLE ANALYSIS
Full DEM observable matrix shape: (1, 274)
Full DEM observable matrix:
Non-zero positions (of 274): [  4  11  12  13  17  18  23  24  25  26  27  32  39  40  44  45  50  51
  52  53  54  55  62  66  67  71  73  74  75  80  81  82  83  84 255]

Our outer DEM observable:
[1 1 0 1 0 1 0 0 0 0 0 0 0]


In [49]:
# Let's compare: what does baseline BPOSD do vs what we're trying?
print("="*70)
print("BASELINE BPOSD vs HIERARCHICAL COMPARISON")
print("="*70)

# BPOSD approach:
# 1. Full DEM: 62 detectors, 274 error vars, 1 observable
# 2. Decode using FULL parity check matrix (62 x 274)
# 3. Compute observable from error pattern using FULL observable matrix (1 x 274)

# Our hierarchical approach:
# 1. Decode inner blocks: 9 blocks × 6 detectors = 54 inner dets
# 2. Get hard/soft decisions per block
# 3. Decode outer: 8 detectors, 13 error vars
# 4. Combine

# The fundamental issue:
# - BPOSD uses ALL 274 error variables and their correlations
# - We try to reduce to 13 outer error vars (9 inner logicals + 4 extras)
# - But the reduction loses critical information!

print("BPOSD uses:")
print(f"  - Full check_matrix: {full_matrices.check_matrix.shape}")
print(f"  - Full observable: {full_matrices.observables_matrix.shape} with {np.sum(full_matrices.observables_matrix.toarray() != 0)} non-zero entries")

print("\nOur outer decoder uses:")
print(f"  - Outer check_matrix: {soft_hier_fixed.outer_matrices.check_matrix.shape}")
print(f"  - Outer observable: {soft_hier_fixed.outer_obs_matrix.shape} with {np.sum(soft_hier_fixed.outer_obs_matrix.toarray() != 0)} non-zero entries")

print("\n*** The hierarchical outer decoder is a GROSS SIMPLIFICATION ***")
print("It can't possibly capture the full error structure!")

# But... the literature says soft hierarchical should work!
# The key is that for PROPER concatenated codes, the structure is cleaner
# Steane + Surface code has complex interactions because both are CSS codes

# What if we try a simpler test case?
print("\n" + "="*70)
print("IDEA: The problem might be code-specific")
print("="*70)
print("Steane code has 6 stabilizers (3X + 3Z)")
print("Surface code has 8 stabilizers (4X + 4Z)")
print("The interaction between them creates complex correlations")
print("")
print("For simpler codes (like repetition inside surface), it might work better.")

BASELINE BPOSD vs HIERARCHICAL COMPARISON
BPOSD uses:
  - Full check_matrix: (62, 274)
  - Full observable: (1, 274) with 35 non-zero entries

Our outer decoder uses:
  - Outer check_matrix: (8, 13)
  - Outer observable: (1, 13) with 4 non-zero entries

*** The hierarchical outer decoder is a GROSS SIMPLIFICATION ***
It can't possibly capture the full error structure!

IDEA: The problem might be code-specific
Steane code has 6 stabilizers (3X + 3Z)
Surface code has 8 stabilizers (4X + 4Z)
The interaction between them creates complex correlations

For simpler codes (like repetition inside surface), it might work better.


# Root Cause Analysis Summary

## Problem
SoftHierarchicalDecoder gives LER ≈ 0.50 (random) when using Template DEM approach, while BPOSD baseline achieves LER ≈ 0.003.

## Root Cause
The outer decoder is fundamentally broken because:

1. **Outer DEM is wrong**: The outer DEM is built with only 8 detectors and 13 error variables, trying to model the "effective channel" where inner blocks contribute logical errors.

2. **Full DEM has 274 error variables**: BPOSD uses the full concatenated DEM with 62 detectors and 274 error variables. The hierarchical approach loses most of this information.

3. **Observable mismatch**: The full observable depends on 35 error variables. Our outer observable only has 4 non-zero entries.

4. **Correlation loss**: The concatenated Steane + Surface code has complex detector-error correlations that span inner and outer boundaries. These are lost in the hierarchical decomposition.

## Why This Happens
The `_build_outer_decoder` method constructs an artificial outer DEM that doesn't match the actual error propagation. It:
- Assumes inner logical errors affect outer stabilizers via `H_outer = outer_code.hz`
- But the actual DEM shows errors affect **both** inner and outer detectors simultaneously
- The hierarchical decomposition assumes independence that doesn't exist

## Possible Fixes

### Option 1: Use TURBO-style iteration
Instead of one-shot inner→outer decode, iterate:
1. Decode inner blocks → soft info
2. Use soft info to condition outer decode
3. Use outer result to re-decode inner
4. Repeat until convergence

### Option 2: Build proper effective outer DEM
Construct the outer DEM properly by:
1. Taking the full DEM
2. Marginalizing out inner detectors (summing over all inner error patterns)
3. This is computationally expensive but correct

### Option 3: Use global soft info without outer decoder
Skip the outer BPOSD decoder entirely:
1. Decode each inner block with Template DEM
2. Combine inner logical decisions using the FULL observable structure
3. This requires knowing the correct combination rule

### Option 4: Accept that hierarchical doesn't work for this code
Some concatenated code combinations may not be amenable to hierarchical decoding. The Steane + Surface combination might have too much cross-talk.